In [177]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
from keras.metrics import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *
from sklearn.neighbors import KNeighborsClassifier
# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing

from sklearn.metrics import *
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

In [2]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

In [205]:
root_dir = './data/tree-rep-profiles_o2o/'
train_pid_paths = pickle.load(open(root_dir + 'train_only_pid_paths.pkl','rb'))
valid_pid_paths = pickle.load(open(root_dir + 'valid_pid_paths.pkl','rb'))
test_pid_paths = pickle.load(open(root_dir + 'test_pid_paths.pkl','rb'))
exp_pid_paths = pickle.load(open(root_dir + 'exp_pid_paths.pkl','rb'))
o2o_pid_paths = pickle.load(open(root_dir + 'o2o_pid_paths.pkl','rb'))

train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train_only.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
o2o_api,o2o_emb,o2o_rep = pickle.load(open(root_dir + 'pids_o2o.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

In [4]:
def basic_statistics(all_length):
    '''
    input: length list of elements
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

model arch與參數設置要與前面一致

In [5]:
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

In [6]:
model_save_path = './model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5' #改
model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc, 'SeqSelfAttention': SeqSelfAttention})
model.load_weights(model_save_path)
model.summary()
thr = 0.482 #0.51

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_10 (Masking)            (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_9 

In [7]:
emb_model = Model(inputs=model.inputs,  outputs=model.layers[5].output)
emb_model.load_weights(model_save_path,by_name=True)
emb_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_10 (Masking)            (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_9 (Masking)             (None, 226, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [856]:
exp_pid_paths[0]

'./data/tree-rep-profiles_o2o/EXP/parite/G0/f2f311d2f3983ce15491aef2d50e036e62681473e85af5278e3b36591df936b5_3424.profile'

## Train
* avg self-vector only in DEV existed's trees
* create important call seq of train data in profile_avg
* No ind REP#: 979
* 4104it [10:44,  6.18it/s]

In [858]:
# train_pid_paths: 每一筆profile路徑位置
# train_emb_api: api embedding vectors
# train_emb: SentVec embedding vectors
#loner: False=>train/dev/test Set ; True: Loner Set
def avg_train_vector(root_dir,thr,train_pid_paths=exp_pid_paths,train_emb_api=exp_api,
                     train_emb=exp_emb,loner=True):
    no_ind_rep = 0
    no_ind_rep_li = []
    tree_vec = []
    empty_hash_li = []
    problem_hash_li = []
    un_aliase_path = './data/tree-rep-profiles_o2o/EXP_rev/'
    if not os.path.exists(root_dir):
        os.makedirs(root_dir,exist_ok=True)
    for path,api,emb in tqdm(zip(train_pid_paths,train_emb_api,train_emb)): #tqdm
        if loner:
            if path not in (claim_path + unk_path):
                continue
            recent_fam = path.split('/')[-3] #fakeav_fakealeart
            recent_hash = path.split('/')[-1].split('_')[0]
            if '_' in recent_fam:
                all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
                recent_fam = all_path_un_aliase[0].split('/')[4]
                
        else:
            recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        hash_id = path.split('/')[-1][:5]
        pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if loner:
            if length < 2:
                empty_hash_li.append(path)
                continue
        else:
            if length < 11:
                print('No common rep. Should not in a tree:',path)
                problem_hash_li.append(path)
                continue
        try:
            assert api[:length][-1] != 0
            assert api[:length+1][-1] == 0
        except AssertionError:
            print('Mask assertion ERR:',path)
            continue
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                no_ind_rep_li.append(path)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    if loner:
        return no_ind_rep_li,empty_hash_li,problem_hash_li
    return no_ind_rep_li

In [801]:
# train/valid/test
root_dir = './data/tree-rep-profiles_o2o/trainingProfile_REP/' #會自動創立資料夾，存放pkl，記得最後的斜線
no_ind_rep = avg_train_vector(root_dir=root_dir,thr=0.482,loner=False)
with open(root_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-3:])+'\n') #-3: fam name、tree id、hash_pid #-1: for loner

4104it [11:59,  5.60it/s]

No ind REP#: 979


In [859]:
# loner
loner_dir = './data/tree-rep-profiles_o2o/lonerProfile_REP/'
no_ind_rep,empty_files,problem_files = avg_train_vector(root_dir=loner_dir,thr=0.482,loner=True)
with open(loner_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-1:])+'\n') #-3: fam name、tree id、hash_pid #-1: for loner
with open(loner_dir + 'empty_file.txt','w') as fp:
    for path in empty_files:
        fp.write(str(path.split('/')[-1:])+'\n') #-3: fam name、tree id、hash_pid #-1: for loner
with open(loner_dir + 'error_file.txt','w') as fp:
    for path in problem_files:
        fp.write(str(path.split('/')[-1:])+'\n') #-3: fam name、tree id、hash_pid #-1: for loner

5it [00:00,  6.86it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
3893it [10:05,  6.03it/s]

No ind REP#: 319


In [795]:
with open(root_dir + 'no_rep_file.txt','w') as fp:
    for path in no_ind_rep:
        fp.write(str(path.split('/')[-3:])+'\n')
no_ind_rep

['./data/tree-rep-profiles_o2o/DEV/18.domaiq_0.8/G397/0dcb0d5c097f7573f2da1453b76021a97029ecde537cd0071bcd7c0a3718672d_3456.profile',
 './data/tree-rep-profiles_o2o/DEV/11.zusy_0.8/G594/8b2e5dd4e0841ef07aa746fa3cf8d74c288b5f440f4ec2a88abb051d1abc82a6_2976.profile',
 './data/tree-rep-profiles_o2o/DEV/11.zusy_0.8/G594/8b2e5dd4e0841ef07aa746fa3cf8d74c288b5f440f4ec2a88abb051d1abc82a6_2968.profile',
 './data/tree-rep-profiles_o2o/DEV/2.zbot_0.8/G1387/5ce3f54a0bfca93b16d015a5681bbbe52b152f769ac934c288cf019ea4c18e9c_3292.profile',
 './data/tree-rep-profiles_o2o/DEV/2.zbot_0.8/G1387/4a275a83c6ce199cc0f01dcf1fa555735668d909bf9738243b0027686d8bc64c_2932.profile',
 './data/tree-rep-profiles_o2o/DEV/2.zbot_0.8/G1387/2f5154d9b42db2141bd2cefa4b93064f35b5d41da4525ebe344f26434ea77f06_3248.profile',
 './data/tree-rep-profiles_o2o/DEV/57.vilsel_0.8/G435/241ac659aac582fd1e93527fa57fe9f35211199075832b332fe8b083a02e10eb_3564.profile',
 './data/tree-rep-profiles_o2o/DEV/57.vilsel_0.8/G435/d1b2142b65d656a477

In [ ]:
no_ind_rep_train_li = []
for i in tqdm(range(1,100)): #system: 500 (0.002)
    thr = i/100
    root_dir = './data/tree-rep-profiles_o2o/profile_avg_exp/'+str(thr)+'/' # data set dir存東西的地方
    no_ind_rep = avg_train_vector(root_dir,thr)
    no_ind_rep_train_li.append(no_ind_rep)

In [265]:
root_dir = './data/tree-rep-profiles_o2o/o2o_all_avg/'
no_ind_rep_train_o2oall = avg_train_vector(root_dir,thr=0.482,train_pid_paths=o2o_pid_paths,
                                           train_emb_api=o2o_api,train_emb=o2o_emb)
len(no_ind_rep_train_o2oall)

0it [00:00, ?it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1259it [03:41,  5.26it/s]

Mask assertion ERR: ./data/tree-rep-profiles_o2o/normal/53.killav_0.8/G98/06c629e39ba32f0e46d79ccabcf345655808e131e585169dbd8ad9b43722f2dd_2924.profile


7222it [21:08,  7.03it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G310/a6736906022a329a383ed668489657f7afce6a2c2babe8faa1f178f646e6b0fa_3352.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G310/564a400c878111f5e320aaf5223775af03d65def7630103e939c7f002d66de81_3368.profile


7226it [21:09,  8.11it/s]

...

10063it [25:54, 10.09it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/small_short/94.bundlore_0.8/G46/16eb6f09233b94724bdd3032d0dc76c9916e5640c3e4f35c5d5669e7f08b312d_2976.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/small_short/94.bundlore_0.8/G46/16eb6f09233b94724bdd3032d0dc76c9916e5640c3e4f35c5d5669e7f08b312d_3320.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/small_short/94.bundlore_0.8/G46/a174a959f8f9cff8813283346fbff0c7839d20fc47d463057562dc8bd6ad0265_3340.profile


10070it [25:56,  6.42it/s]

No ind REP#: 1425


1425

## DEV/Test
* cos-sim of each individual rep
* TopN (cos-sim>0.8)
* 每個pid歸類到的tree_name一樣就成功

In [17]:
def prepare_inf_data(ind_rep_dir):
# ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/' #要有最後的反斜線
    ind_reps = next(os.walk(ind_rep_dir))[2]
    ind_rep_tree = []
    ind_rep_fam = []
    for i,c in enumerate(ind_reps):
        ind_rep = pickle.load(open(ind_rep_dir + c,'rb')) #每個tree的centroid pickle
        if i==0:
            ind_reps_matrix = ind_rep # 1st time
        else:
            ind_reps_matrix = np.concatenate((ind_reps_matrix,ind_rep),axis=0) #依順序合在一起 
        ind_rep_tree.append(c.split('_')[1])
        ind_rep_fam.append(c.split('_')[0].split('.')[-1])
    assert ind_reps_matrix.shape[0] == len(ind_rep_tree) == len(ind_rep_fam)
    return ind_reps_matrix,ind_rep_tree,ind_rep_fam

In [691]:
def EM_index(score):
    '''
    input: cosin sim 分數
    output: 同分的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) # 取道小數點地五為四捨五入到第四位
    em_id = []
    for s in scores:
        now_score = np.round(score[0][s],decimals=4) #現在這個pairwise 的similarity
        if now_score ==tmp_score: #同分增額錄取
            em_id.append(s)
    return em_id,tmp_score

In [692]:
def inf_evaluate(inf_pid_paths,inf_emb_api,inf_emb,ind_reps_matrix,ind_rep_tree,ind_rep_fam,thr=thr):
    em_up = 0
    no_ind_rep = 0
    bound_accuracy = []
    wrong_tree = {}
    wrong_fam = {}
    REP_len = []
    all_length = []
    family_match = [] # 分對家族 1 否則-1 # 僅限有REP者
    family_nmatch = [] # 分錯家族 2 否則-1
    tree_match = [] # 分對BT 3 否則-1
    tree_nmatch = [] # 分錯BT 4 否則-1
    fam_li = [] #ground truth of fam
    all_predict_fam = []
    tree_li = [] #ground truth of tree
    all_predict_tree = []
    EM_sim_score = []
    
    family_match_all = [] # 分對家族 1 否則-1
    family_nmatch_all = [] # 分錯家族 2 否則-1
    tree_match_all = [] # 分對BT 3 否則-1
    tree_nmatch_all = [] # 分錯BT 4 否則-1    
    no_rep_list= []
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)): #tqdm
        recent_tree = path.split('/')[-2]
        recent_fam = path.split('/')[-3].split('_')[0].split('.')[-1] #EX: shodi
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        rep_length = sum(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                no_rep_list.append(path)
                family_match_all.append(-1) # 分對家族 1 否則-1
                family_nmatch_all.append(2)  # 分錯家族 2 否則-1
                tree_match_all.append(-1) # 分對BT 3 否則-1
                tree_nmatch_all.append(4)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        
        REP_len.append(rep_length)
        all_length.append(length)
        
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        score = cosine_similarity(rep_emb_avg,ind_reps_matrix)
        
        scores = score[0].argsort() #相似度小到大
        scores = np.flip(scores) # 相似度大到小
        for idx in scores:
            if ind_rep_tree[idx] == recent_tree:
                bound_accuracy.append(score[0][idx]) #最差都是幾分的時候合併
        
        idx_list,max_score = EM_index(score) # exact match的id
        EM_sim_score.append(max_score)
        em_tree = []
        for idx in idx_list:
            em_tree.append(ind_rep_tree[idx])
        em_tree = list(set(em_tree)) # exact match的tree name
        all_predict_tree.append(em_tree)
        tree_li.append(recent_tree)
        if recent_tree in em_tree: #tree match
            em_up = em_up + 1
            tree_match.append(3) #畫圖
            tree_nmatch.append(-1) # 畫圖
            tree_match_all.append(3) # 分對BT 3 否則-1
            tree_nmatch_all.append(-1)            
        else: #tree not match
            tree_nmatch.append(4)
            tree_match.append(-1)
            tree_match_all.append(-1) # 分對BT 3 否則-1
            tree_nmatch_all.append(4)         
            wrong_tree[path]=em_tree
            
        em_fam = []
        for idx in idx_list:
            em_fam.append(ind_rep_fam[idx])
        em_fam = list(set(em_fam)) # exact match的tree name
        all_predict_fam.append(em_fam)
        fam_li.append(recent_fam)
        if recent_fam in em_fam: # family match
            family_match.append(1) #畫圖
            family_nmatch.append(-1) # 畫圖
            family_match_all.append(1) # 分對家族 1 否則-1
            family_nmatch_all.append(-1)
            
        else: # family not match
            family_nmatch.append(2) #畫圖
            family_match.append(-1)      # 畫圖   
            family_match_all.append(-1) # 分對家族 1 否則-1
            family_nmatch_all.append(2)
            wrong_fam[path] = em_fam
    down = len(inf_pid_paths)
#     print('ExactMatchAccuracyScore:',em_up/down,'EM#:/all#:/NoIndREP#:',em_up,down,no_ind_rep,
#          'ExactMatchPrecision(w/o No ind REP):',em_up/(down-no_ind_rep)
#          )
    return (bound_accuracy,wrong_fam,wrong_tree,em_up/down,no_ind_rep,em_up/(down-no_ind_rep),sum(REP_len)/len(REP_len),sum(all_length)/len(all_length),
           family_match_all,family_nmatch_all,tree_match_all,tree_nmatch_all,family_match,family_nmatch,tree_match,tree_nmatch,
            fam_li,tree_li,all_predict_fam,all_predict_tree,EM_sim_score,no_rep_list) #真正正確的tree是在幾%相似度才有? 、{[正確 path]=predict list}

In [693]:
# thr_test
thr_li = []
EM_score_li = []
no_rep_num_li = []
EMw_score_li = []
rep_avg_len = []
profile_avg_len = []
for i in tqdm(range(1,100)): #system: 500 (0.002)
    thr = i/100
    ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg_exp/'+str(thr)+'/' # data set dir
    matrix,tree_ans,fam_ans = prepare_inf_data(ind_rep_dir)
    temp1,temp2,EM_score,no_rep,EMw_score,avg_rep,avg_profile = inf_evaluate(test_pid_paths,test_emb_api,test_emb
                                                         ,ind_reps_matrix=matrix,ind_rep_tree=tree_ans,thr=thr)
    thr_li.append(thr)
    EM_score_li.append(EM_score)
    no_rep_num_li.append(no_rep)
    EMw_score_li.append(EMw_score)
    rep_avg_len.append(avg_rep)
    profile_avg_len.append(avg_profile)

  0%|          | 0/99 [00:00<?, ?it/s]


TypeError: inf_evaluate() missing 1 required positional argument: 'ind_rep_fam'

In [227]:
score_df = pd.DataFrame(data={'threshold':thr_li,'EM_acc':EM_score_li,'No_REP':no_rep_num_li,'EMw_acc':EMw_score_li
                             ,'REP_avg_len':rep_avg_len,'Profile_avg_len':profile_avg_len})
score_df.to_excel('data/tree-rep-profiles_o2o/threxp_EM_lenratio.xlsx',index=False)
score_df

,threshold,EM_acc,No_REP,EMw_acc,REP_avg_len,Profile_avg_len
0,0.01,0.952756,10,0.971888,67.574297,74.437751
1,0.02,0.940945,14,0.967611,65.372470,74.623482
2,0.03,0.929134,17,0.961303,63.560081,74.798371
3,0.04,0.927165,23,0.971134,62.764948,74.989691
4,0.05,0.925197,23,0.969072,62.270103,74.989691
5,0.06,0.919291,23,0.962887,61.983505,74.989691
6,0.07,0.921260,23,0.964948,61.630928,74.989691
7,0.08,0.925197,23,0.969072,61.435052,74.989691
8,0.09,0.923228,23,0.967010,61.268041,74.989691
9,0.10,0.917323,28,0.970833,61.652083,75.083333


In [258]:
score_df[score_df['threshold']== 0.51]

,threshold,EM_acc,No_REP,EMw_acc,REP_avg_len,Profile_avg_len
50,0.51,0.732283,125,0.971279,52.678851,73.446475


畫圖

In [694]:
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/'#_exp/'+str(thr)+'/' # data set dir
matrix,tree_ans,fam_ans = prepare_inf_data(ind_rep_dir)
(temp1,test_wrong_fam,test_wrong_tree,EM_score,no_rep,EMw_score,avg_rep,avg_profile,
FM_all,FN_all,TM_all,TN_all,FM,FN,TM,TN,
 fam_truth,tree_truth,fam_predict,tree_predict,EM_score_li,no_rep_path) = inf_evaluate(test_pid_paths,test_emb_api,test_emb
                                                     ,ind_reps_matrix=matrix,ind_rep_tree=tree_ans,
                                                                         ind_rep_fam=fam_ans,thr=0.482)

5it [00:00,  5.81it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
11it [00:01,  6.06it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
29it [00:04,  6.18it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret 

376it [00:58,  6.82it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
385it [01:00,  6.46it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
393it [01:01,  6.69it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(

In [702]:
test_mismatch_fam_pd = pd.DataFrame(columns=['ori_fam','ori_tree','sha256','pid','predict_fam'])
for i , (path,pred) in enumerate(test_wrong_fam.items()):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
    tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    test_mismatch_fam_pd.loc[i] = [fam,tree,sha256,pid,pred]
test_mismatch_fam_pd

,ori_fam,ori_tree,sha256,pid,predict_fam
0,zbot,G1387,048a0153d4ff0ebc84b4fc28e2a0d5adfc107058359ff3...,3216,"[fakeav, vilsel]"
1,zbot,G1365,1f98532f7d2d738722f65790f51af0ae40c5d7559643d2...,2796,[fakeav]
2,sytro,G481,02c6e7a9ebf264e98c75c0ffe0498437937b9bffd5854e...,2956,"[zbot, virut, allaple]"
3,virut,G1440,d9bda6a10c7e2ad4eb4f717fbcbac9077012d5029aac8d...,3284,"[allaple, sytro]"
4,fakeav,G2370,4ba1b86d5a13b5b63e43bb2e6d7dde12bffe60b76fc61c...,3264,[zbot]
5,fakeav,G2349,0b7205b3c948334bbdb5ea566f494b77bea19ad8c2b261...,3328,"[ursu, symmi, zbot]"
6,fakealert,G558,79d3d697df8d4a1522c0fbe25ef72e52a1357ea9cd1401...,3276,"[fakeav, zbot]"
7,symmi,G663,00a35edc0afc06d4c787cea687f9eabf82c953b1a218c2...,2900,[vobfus]
8,allaple,G2100,df2ed7a1586129f5278449407bb84421c2903818ca7c24...,2932,[medfos]


In [703]:
test_mismatch_fam_pd.to_excel('./data/tree-rep-profiles_o2o/test_mismatch_fam_sha256.xlsx')

In [704]:
test_mismatch_tree_pd = pd.DataFrame(columns=['ori_fam','ori_tree','sha256','pid','predict_tree'])
for i , (path,pred) in enumerate(test_wrong_tree.items()):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
    tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    test_mismatch_tree_pd.loc[i] = [fam,tree,sha256,pid,pred]
test_mismatch_tree_pd

,ori_fam,ori_tree,sha256,pid,predict_tree
0,zbot,G1387,048a0153d4ff0ebc84b4fc28e2a0d5adfc107058359ff3...,3216,"[G429, G2370]"
1,zbot,G1365,1f98532f7d2d738722f65790f51af0ae40c5d7559643d2...,2796,[G2332]
2,vilsel,G436,7c407183d74e8373a567da381b0d563682ebb1a2b9c0ab...,3424,[G429]
3,sytro,G481,02c6e7a9ebf264e98c75c0ffe0498437937b9bffd5854e...,2956,"[G1445, G2100, G1390]"
4,virut,G1440,d9bda6a10c7e2ad4eb4f717fbcbac9077012d5029aac8d...,3284,"[G2082, G485]"
5,fakeav,G2370,4ba1b86d5a13b5b63e43bb2e6d7dde12bffe60b76fc61c...,3264,[G1387]
6,fakeav,G2349,0b7205b3c948334bbdb5ea566f494b77bea19ad8c2b261...,3328,"[G669, G1379, G1055]"
7,fakealert,G558,79d3d697df8d4a1522c0fbe25ef72e52a1357ea9cd1401...,3276,"[G1386, G2333]"
8,symmi,G663,00a35edc0afc06d4c787cea687f9eabf82c953b1a218c2...,2900,[G462]
9,allaple,G2100,df2ed7a1586129f5278449407bb84421c2903818ca7c24...,2932,[G82]


In [705]:
test_mismatch_tree_pd.to_excel('./data/tree-rep-profiles_o2o/test_mismatch_tree_sha256.xlsx')

In [676]:
test_no_rep_pd = pd.DataFrame(columns=['fam','tree','sha256','pid'])
for i,path in enumerate(no_rep_path):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
    tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    test_no_rep_pd.loc[i] = [fam,tree,sha256,pid]
test_no_rep_pd

,fam,tree,sha256,pid
0,domaiq,G397,0dcb0d5c097f7573f2da1453b76021a97029ecde537cd0...,3464
1,zusy,G594,019a496fee9eff1fc2d428982415aefd100bba5c593d7e...,2996
2,zbot,G1390,1c94be5cb0a57a4bf88e09470cbb069e0478efab7bb2c1...,3300
3,zbot,G1390,3bc6b9292c6b64b83aed656ed554975004296631b3265a...,3360
4,zbot,G1387,8a387c734c5f4df1591665ce2d33c5d8e3be7672bc8071...,3252
5,zbot,G1387,0cc9db8c027d7a0d4b4012db214a93c7be6a3ea2076810...,3356
6,vilsel,G435,7c196c432432be3a495a24edfaacb5f638c033ede8c6b2...,3384
7,vilsel,G435,7fbbc00fb1503da75926543c6cbe01bf15eb9370113da9...,3208
8,vilsel,G435,02ab72ab1a987f264256cf1fd8399df101bf6703bfae95...,3448
9,virut,G1429,f1582922a0c97b8b8dc578430b49af3f24c6968025ef71...,3392


In [677]:
test_no_rep_pd.to_excel('./data/tree-rep-profiles_o2o/test_no_rep_hash.xlsx')

In [54]:
# all test set draw
number_list = []
fam_all_list = []
for i,v in enumerate(test_pid_paths):
    number_list.append(i+1)
    fam_all_list.append(v.split('/')[-3].split('_')[0].split('.')[-1])

In [55]:
draw_df = pd.DataFrame(data={'true_fam':fam_all_list,'numberID':number_list,'FamMatch_all':FM_all,'FamNMatch_all':FN_all
                             ,'TreeMatch_all':TM_all,'TreeNMatch_all':TN_all})
draw_df.to_excel('data/tree-rep-profiles_o2o/EM_draw_allTest.xlsx',index=False)
draw_df

,true_fam,numberID,FamMatch_all,FamNMatch_all,TreeMatch_all,TreeNMatch_all
0,domaiq,1,1,-1,3,-1
1,domaiq,2,1,-1,3,-1
2,domaiq,3,1,-1,3,-1
3,domaiq,4,1,-1,3,-1
4,domaiq,5,1,-1,3,-1
5,domaiq,6,-1,2,-1,4
6,ipamor,7,1,-1,3,-1
7,ipamor,8,1,-1,3,-1
8,ipamor,9,1,-1,3,-1
9,lydra,10,1,-1,3,-1


In [44]:
# w/o no REP draw
number_list = [] 
fam_list = []
for i,v in enumerate(fam_truth):
    number_list.append(i+1)
fam_truth

['domaiq',
 'domaiq',
 'domaiq',
 'domaiq',
 'domaiq',
 'ipamor',
 'ipamor',
 'ipamor',
 'lydra',
 'zusy',
 'zusy',
 'zusy',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'zbot',
 'autoit',
 'autoit',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'vilsel',
 'browsefox',
 'browsefox',
 'browsefox',
 'browsefox',
 'browsefox',
 'ramnit',
 'ramnit',
 'sality',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'sytro',
 'medfos',
 'virut',
 'virut',
 'virut',
 'virut',
 'virut',
 'virut',
 'virut',
 'virut',
 'virut',
 'virut',
 'downloa',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 'ursu',
 '

In [49]:
draw_df = pd.DataFrame(data={'true_fam':fam_truth,'numberID':number_list,'FamMatch':FM,'FamNMatch':FN
                             ,'TreeMatch':TM,'TreeNMatch_all':TN})
draw_df.to_excel('data/tree-rep-profiles_o2o/EM_draw_onlywREP.xlsx',index=False)
draw_df

,true_fam,numberID,FamMatch,FamNMatch,TreeMatch,TreeNMatch_all
0,domaiq,1,1,-1,3,-1
1,domaiq,2,1,-1,3,-1
2,domaiq,3,1,-1,3,-1
3,domaiq,4,1,-1,3,-1
4,domaiq,5,1,-1,3,-1
5,ipamor,6,1,-1,3,-1
6,ipamor,7,1,-1,3,-1
7,ipamor,8,1,-1,3,-1
8,lydra,9,1,-1,3,-1
9,zusy,10,1,-1,3,-1


In [48]:
set(fam_all_list) - set(fam_truth)

{'graftor', 'sirefef'}

#### argMax dist. analyze
* F1 / precision / recall
* fam / tree predicted nums
* EM_score dist.

In [186]:
fam_truth_li = []
for fam in fam_truth:
    fam_truth_li.append([fam])
tree_truth_li = []
for tree in tree_truth:
    tree_truth_li.append([tree])
assert len(fam_truth_li) == len(tree_truth_li)

In [187]:
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(fam_truth_li)
y_pred = mlb.transform(fam_predict)
pre = precision_score(y_true,y_pred,average='micro')
rec = recall_score(y_true,y_pred,average='micro')
f1 = f1_score(y_true,y_pred,average='micro')
print('Family precision/recall/f1',pre,rec,f1)
mlb = MultiLabelBinarizer()
y_pred = mlb.fit_transform(tree_predict)
y_true = mlb.transform(tree_truth_li)
pre = precision_score(y_true,y_pred,average='micro')
rec = recall_score(y_true,y_pred,average='micro')
f1 = f1_score(y_true,y_pred,average='micro')
print('BT precision/recall/f1',pre,rec,f1)

Family precision/recall/f1 0.1831610044313146 0.9712793733681462 0.30820215410107704
BT precision/recall/f1 0.16956124314442414 0.9686684073107049 0.2886036561649164


In [188]:
fam_predict_num = []
for fam in fam_predict:
    fam_predict_num.append(len(fam))
basic_statistics(fam_predict_num)

,length
mean,5.464752
std,5.042191
mode,1.000000
min,1.000000
q1,1.000000
median,4.000000
q3,6.000000
max,18.000000
iqr,5.000000
outlier,13.500000


In [189]:
tree_predict_num = []
for tree in tree_predict:
    tree_predict_num.append(len(tree))
basic_statistics(tree_predict_num)

,length
mean,5.712794
std,5.430406
mode,1.000000
min,1.000000
q1,1.000000
median,4.000000
q3,6.000000
max,18.000000
iqr,5.000000
outlier,13.500000


In [184]:
basic_statistics(EM_score_li) #decimal=7

,length
mean,0.999958
std,0.000655
mode,1.000000
min,0.987315
q1,1.000000
median,1.000000
q3,1.000000
max,1.000000
iqr,0.000000
outlier,1.000000


In [375]:
kk = dict(Counter(fam_predict_num))
argmax_fam_num_left = list(kk.keys())
argmax_fam_num_right = list(kk.values())

In [377]:
match_df = pd.DataFrame(data={'argmax_fam_num':fam_predict_num,'argmax_tree_num':tree_predict_num
                             ,'EM_score':EM_score_li})
match_df.to_excel('data/tree-rep-profiles_o2o/argmax_match_dist.xlsx',index=False)
match_df

ValueError: arrays must all be same length

In [379]:
match_df = pd.DataFrame(data={'left':argmax_fam_num_left, 'right':argmax_fam_num_right})
match_df.to_excel('data/tree-rep-profiles_o2o/argmax_match_fam_dist.xlsx',index=False)
match_df

,left,right
0,12,45
1,15,25
2,18,17
3,2,21
4,3,40
5,1,111
6,6,48
7,4,69
8,5,7


#### UNK analysis
* training unk (得知全沒、部分沒REP)
    * normal中的tree、fam
    * avg_profile中的tree、fam
    * normal - avg_profile : 全沒
* testing unk (是否落在部分沒REP中)
    * tree、fam

In [321]:
# 直接從資料夾來判斷原始 fam 跟 tree
train_dir = './data/tree-rep-profiles_o2o/normal/'
train_fam = next(os.walk(train_dir))[1]
train_tree = []
fam_profiles_nums = {}
tree_profiles_nums = {}
for fam in train_fam:
    tree_dir = next(os.walk(train_dir +fam))[1]
    train_tree.extend(tree_dir)
    fam_profile_count = 0
    for tree in tree_dir:
        in_directory = train_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        tree_profile_count = len(hl_list)
        tree_profiles_nums[fam.split('_')[0] + '_' + tree] = tree_profile_count
        fam_profile_count = fam_profile_count + tree_profile_count
    fam_profiles_nums[fam.split('_')[0]] = fam_profile_count
train_fam = [x.split('_')[0] for x in train_fam]

In [329]:
train_fam = list(set(list(fam_profiles_nums.keys()) )-set(train_only_fam)) #來自下面TEST only 47
train_tree = list(set(list(tree_profiles_nums.keys()))-set(train_only_tree)) #要先跑下面在重新跑這一格，跟下面重跑
len(train_fam) , len(train_tree)

(86, 673)

In [320]:
train_fam = list(fam_profiles_nums.keys()) 
train_tree = list(tree_profiles_nums.keys())
len(train_fam) , len(train_tree)

(133, 808)

In [331]:
o2o_avg_dir = './data/tree-rep-profiles_o2o/o2o_all_avg/'
files = next(os.walk(o2o_avg_dir))[2]
claim_avg_paths = []
claim_avg_fam = []
avg_tree = []
claim_avg_tree = []
for file in files:
    fam = file.split('_')[0]
    if fam not in train_fam:
#         print(fam)
        continue
    tree = fam+'_'+file.split('_')[1]
    if tree not in train_tree:
        continue
    claim_avg_paths.append(file)
    claim_avg_fam.append(fam)
    
    claim_avg_tree.append(tree)
print('133 family中有rep的profiles個數:',len(set(claim_avg_paths)))
print('133 family 中剩下 fam/tree個數:',len(set(claim_avg_fam)),len(set(claim_avg_tree))) #相減就是全沒的數量
avg_fam_profiles_nums = dict(Counter(claim_avg_fam))
avg_tree_profiles_nums = dict(Counter(claim_avg_tree))

133 family中有rep的profiles個數: 790
133 family 中剩下 fam/tree個數: 81 191


In [282]:
# print('133中全沒fam',len(set(train_fam)-set(claim_avg_fam)))
# print('133中全沒tree:',len(set(train_tree)-set(claim_avg_tree)))

133中全沒fam 2
133中全沒tree: 351


In [295]:
happen_no_fam = []
for k,v in avg_fam_profiles_nums.items():
    if fam_profiles_nums[k] == v:
        continue
    happen_no_fam.append(k)
print('有發生NO ind REP的家族#:',len(set(happen_no_fam)))
happen_no_tree = []
for k,v in avg_tree_profiles_nums.items():
    if tree_profiles_nums[k] == v:
        continue
    happen_no_tree.append(k)
print('有發生NO ind REP的BT#:',len(set(happen_no_tree)))

有發生NO ind REP的家族#: 62
有發生NO ind REP的BT#: 80


only in test

In [299]:
train_only_fam = []
# train_only_fam_nums = {}
train_only_tree = []
# train_only_tree_nums = {}
print('在47個fam的train set中共有#個profiles',len(train_pid_paths))
for train_only_path in train_pid_paths:
    fam = train_only_path.split('/')[4].split('_')[0]
    tree = fam + '_' + train_only_path.split('/')[5]
    train_only_fam.append(fam)
    train_only_tree.append(tree)
train_only_fam_nums = dict(Counter(train_only_fam))
train_only_tree_nums = dict(Counter(train_only_tree))

在47個fam的train set中共有#個profiles 4104


In [305]:
# 有train rep的資料夾
avg_dir = './data/tree-rep-profiles_o2o/profile_avg/'
files = next(os.walk(avg_dir))[2]
avg_fam = []
avg_tree = []
print('在47個fam的train set中有#個profiles才有REP',len(files))
for file in files:
    fam = file.split('_')[0]
    avg_fam.append(fam)
    tree = fam +'_' +file.split('_')[1]
    avg_tree.append(tree)
avg_fam_nums = dict(Counter(avg_fam))
avg_tree_nums = dict(Counter(avg_tree))

在47個fam的train set中有#個profiles才有REP 3125


In [318]:
print('在47個家族中有多少個fam完全沒有rep',len(set(train_only_fam) - set(avg_fam))) #在47個家族中有多少個fam完全沒有rep
print('在47個家族中有多少個tree完全沒有rep',len(set(train_only_tree) - set(avg_tree)))
set(train_only_fam) - set(avg_fam) , set(train_only_tree) - set(avg_tree)

在47個家族中有多少個fam完全沒有rep 1
在47個家族中有多少個tree完全沒有rep 8


({'39.sirefef'},
 {'1.fakeav_G2265',
  '1.fakeav_G2277',
  '13.vobfus_G459',
  '37.somoto_G220',
  '39.sirefef_G120',
  '4.kazy_G1359',
  '8.fakealert_G548',
  '8.fakealert_G563'})

In [307]:
happen_no_fam_train_only = []
for k,v in avg_fam_nums.items():
    if train_only_fam_nums[k] == v:
        continue
    happen_no_fam_train_only.append(k)
print('有發生NO ind REP的家族#:',len(set(happen_no_fam_train_only)))
happen_no_tree_train_only = []
for k,v in avg_tree_nums.items():
    if train_only_tree_nums[k] == v:
        continue
    happen_no_tree_train_only.append(k)
print('有發生NO ind REP的BT#:',len(set(happen_no_tree_train_only)))

有發生NO ind REP的家族#: 21
有發生NO ind REP的BT#: 25


In [313]:
happen_no_tree_train_only

['2.zbot_G1390',
 '1.fakeav_G2363',
 '7.solimba_G762',
 '1.fakeav_G2368',
 '10.symmi_G668',
 '3.allaple_G2100',
 '18.domaiq_G395',
 '24.softpulse_G339',
 '11.zusy_G594',
 '4.kazy_G1362',
 '16.sytro_G481',
 '6.virut_G1445',
 '11.zusy_G592',
 '18.domaiq_G397',
 '15.graftor_G447',
 '1.fakeav_G2370',
 '22.razy_G281',
 '6.virut_G1429',
 '8.fakealert_G556',
 '2.zbot_G1387',
 '12.expiro_G641',
 '25.browsefox_G327',
 '58.picsys_G115',
 '57.vilsel_G435',
 '32.outbrowse_G267']

In [310]:
len((set(no_rep_path)))

125

In [335]:
# avg_dir = './data/tree-rep-profiles_o2o/profile_avg/'
train_pid_paths[0]

'./data/tree-rep-profiles_o2o/normal/18.domaiq_0.8/G395/b1debd162b5f21c1088b12f9959a87c4a23d4590b0daf5768c84a873c2533dd6_3368.profile'

In [380]:
unk_ori_fam_num = []
unk_fam_have_rep_num = []
unk_ori_bt_num = []
unk_bt_have_rep_num = []
ori_fam = []
ori_bt = []
for path in no_rep_path:
    fam = path.split('/')[4].split('_')[0]
    tree = path.split('/')[5]
    ori_fam.append(fam)
    ori_bt.append(fam + '_'+tree)
    fam_have_rep = glob.glob(avg_dir+fam+'*') #have
    tree_have_rep = glob.glob(avg_dir+fam+'_'+tree+'*')  #have
    fam_all_num = train_only_fam_nums[fam] #ori
    tree_all_num = train_only_tree_nums[fam + '_'+tree] #ori
    unk_ori_fam_num.append(fam_all_num)
    unk_fam_have_rep_num.append(len(fam_have_rep))
    unk_ori_bt_num.append(tree_all_num)
    unk_bt_have_rep_num.append(len(tree_have_rep))
unk_bt_have_rep_num

[5,
 8,
 47,
 47,
 1,
 1,
 1,
 1,
 1,
 4,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 3,
 3,
 0,
 12,
 12,
 12,
 0,
 0,
 24,
 24,
 24,
 24,
 24,
 5,
 5,
 6,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 2,
 2,
 9,
 9,
 9,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93,
 93]

In [386]:
kk = dict(Counter(ori_fam))
fam_name = list(kk.keys())
fam_name = [x.split('.')[1] for x in fam_name]
fam_count = list(kk.values())

In [387]:
unk_fam_df = pd.DataFrame(data={'left':fam_name, 'right':fam_count})
unk_fam_df.to_excel('data/tree-rep-profiles_o2o/argmax_unk_fam_dist.xlsx',index=False)
unk_fam_df

,left,right
0,domaiq,1
1,zusy,1
2,zbot,4
3,vilsel,3
4,virut,8
5,outbrowse,2
6,vobfus,1
7,graftor,3
8,kazy,7
9,solimba,2


In [367]:
unk_ana = pd.DataFrame(data={'Ori_Fam':ori_fam,'Ori_BT':ori_bt,'Ori_Fam_profileNum':unk_ori_fam_num,
                             'TrainFam_Have_REP_num':unk_fam_have_rep_num,'Ori_BT_profileNum':unk_ori_bt_num,
                             'TrainBT_Have_REP_num':unk_bt_have_rep_num})
unk_ana.to_excel('data/tree-rep-profiles_o2o/unk_noREP_analyze.xlsx',index=False)
unk_ana

,Ori_Fam,Ori_BT,Ori_Fam_profileNum,TrainFam_Have_REP_num,Ori_BT_profileNum,TrainBT_Have_REP_num
0,18.domaiq,18.domaiq_G397,49,36,14,5
1,11.zusy,11.zusy_G594,38,25,20,8
2,2.zbot,2.zbot_G1390,253,225,51,47
3,2.zbot,2.zbot_G1390,253,225,51,47
4,2.zbot,2.zbot_G1387,253,225,25,1
5,2.zbot,2.zbot_G1387,253,225,25,1
6,57.vilsel,57.vilsel_G435,117,94,24,1
7,57.vilsel,57.vilsel_G435,117,94,24,1
8,57.vilsel,57.vilsel_G435,117,94,24,1
9,6.virut,6.virut_G1429,139,109,13,4


In [368]:
have_rep_path = (set(test_pid_paths) - set(no_rep_path))
known_ori_fam_num = []
known_fam_have_rep_num = []
known_ori_bt_num = []
known_bt_have_rep_num = []
ori_fam = []
ori_bt = []
for path in have_rep_path:
    fam = path.split('/')[4].split('_')[0]
    tree = path.split('/')[5]
    ori_fam.append(fam)
    ori_bt.append(fam + '_'+tree)
    fam_have_rep = glob.glob(avg_dir+fam+'*') #have
    tree_have_rep = glob.glob(avg_dir+fam+'_'+tree+'*')  #have
    fam_all_num = train_only_fam_nums[fam] #ori
    tree_all_num = train_only_tree_nums[fam + '_'+tree] #ori
    known_ori_fam_num.append(fam_all_num)
    known_fam_have_rep_num.append(len(fam_have_rep))
    known_ori_bt_num.append(tree_all_num)
    known_bt_have_rep_num.append(len(tree_have_rep))
known_bt_have_rep_num

[131,
 270,
 93,
 270,
 33,
 36,
 124,
 63,
 36,
 124,
 166,
 93,
 32,
 2,
 131,
 14,
 34,
 18,
 27,
 270,
 68,
 12,
 32,
 17,
 270,
 17,
 49,
 16,
 93,
 21,
 21,
 9,
 131,
 26,
 166,
 32,
 32,
 18,
 68,
 48,
 124,
 270,
 19,
 27,
 32,
 2,
 36,
 124,
 38,
 13,
 19,
 37,
 124,
 93,
 38,
 166,
 15,
 63,
 270,
 16,
 270,
 13,
 13,
 12,
 12,
 270,
 37,
 114,
 38,
 37,
 16,
 131,
 9,
 38,
 49,
 16,
 63,
 114,
 93,
 93,
 68,
 31,
 12,
 27,
 68,
 166,
 270,
 17,
 22,
 131,
 24,
 14,
 36,
 270,
 38,
 166,
 48,
 49,
 114,
 49,
 38,
 6,
 11,
 131,
 38,
 12,
 13,
 12,
 57,
 38,
 14,
 32,
 12,
 20,
 113,
 270,
 166,
 10,
 48,
 14,
 12,
 270,
 270,
 68,
 9,
 63,
 49,
 47,
 62,
 50,
 9,
 20,
 114,
 32,
 114,
 10,
 113,
 93,
 19,
 166,
 131,
 63,
 9,
 8,
 49,
 12,
 38,
 33,
 12,
 124,
 124,
 48,
 10,
 270,
 9,
 38,
 166,
 21,
 114,
 37,
 13,
 16,
 62,
 113,
 270,
 10,
 16,
 166,
 47,
 124,
 25,
 124,
 49,
 34,
 8,
 68,
 166,
 113,
 9,
 13,
 13,
 9,
 2,
 9,
 20,
 270,
 270,
 10,
 49,
 114,
 270,
 93,


In [370]:
known_ana = pd.DataFrame(data={'Ori_Fam':ori_fam,'Ori_BT':ori_bt,'Ori_Fam_profileNum':known_ori_fam_num,
                             'TrainFam_Have_REP_num':known_fam_have_rep_num,'Ori_BT_profileNum':known_ori_bt_num,
                             'TrainBT_Have_REP_num':known_bt_have_rep_num})
known_ana.to_excel('data/tree-rep-profiles_o2o/known_wREP_analyze.xlsx',index=False)
known_ana

,Ori_Fam,Ori_BT,Ori_Fam_profileNum,TrainFam_Have_REP_num,Ori_BT_profileNum,TrainBT_Have_REP_num
0,5.ursu,5.ursu_G1057,257,257,131,131
1,3.allaple,3.allaple_G2097,719,392,270,270
2,3.allaple,3.allaple_G2100,719,392,420,93
3,3.allaple,3.allaple_G2097,719,392,270,270
4,14.hotbar,14.hotbar_G407,60,60,33,33
5,54.fesber,54.fesber_G373,61,61,36,36
6,16.sytro,16.sytro_G485,133,127,124,124
7,62.directdow,62.directdow_G196,85,85,63,63
8,54.fesber,54.fesber_G373,61,61,36,36
9,16.sytro,16.sytro_G485,133,127,124,124


In [656]:
no_rep_path

['./data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G397/0dcb0d5c097f7573f2da1453b76021a97029ecde537cd0071bcd7c0a3718672d_3464.profile',
 './data/tree-rep-profiles_o2o/TEST/11.zusy_0.8/G594/019a496fee9eff1fc2d428982415aefd100bba5c593d7e5b4dff214c6f68ff34_2996.profile',
 './data/tree-rep-profiles_o2o/TEST/2.zbot_0.8/G1390/1c94be5cb0a57a4bf88e09470cbb069e0478efab7bb2c1f9908838944f0a4e86_3300.profile',
 './data/tree-rep-profiles_o2o/TEST/2.zbot_0.8/G1390/3bc6b9292c6b64b83aed656ed554975004296631b3265acb75d3acae4a990143_3360.profile',
 './data/tree-rep-profiles_o2o/TEST/2.zbot_0.8/G1387/8a387c734c5f4df1591665ce2d33c5d8e3be7672bc8071c7392f4be3009a0a2a_3252.profile',
 './data/tree-rep-profiles_o2o/TEST/2.zbot_0.8/G1387/0cc9db8c027d7a0d4b4012db214a93c7be6a3ea20768106e7b30a9e71109022a_3356.profile',
 './data/tree-rep-profiles_o2o/TEST/57.vilsel_0.8/G435/7c196c432432be3a495a24edfaacb5f638c033ede8c6b25e28811e39693d2bcc_3384.profile',
 './data/tree-rep-profiles_o2o/TEST/57.vilsel_0.8/G435/7fbbc00fb1

In [660]:
# 把沒有REP的要分析家族複製到另外一個資料夾
analyze_dest = './data/tree-rep-profiles_o2o/analyze_noREP/Loner/' #改
analyze_fam = 'allaple'
out_dir = analyze_dest+analyze_fam+'/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir,exist_ok=True)
for path in tqdm(unk_path): #改 #unk_path #no_rep_path
    if analyze_fam in path:
        profile_name = path.split('/')[-1]
        shutil.copy(path,out_dir+profile_name)

100%|██████████| 319/319 [00:00<00:00, 91261.37it/s]


In [ ]:
# 從經過處理的path來判斷原始 fam跟tree


In [203]:
len(set(avg_fam))

46

In [202]:
set(train_fam) - set(avg_fam)

{'100.jadtre',
 '101.banker',
 '103.fynloski',
 '105.inffinity',
 '106.gator',
 '108.zegost',
 '109.bredo',
 '111.imali',
 '112.installmonetizer',
 '113.ipz',
 '115.alman',
 '116.lethic',
 '118.asoecem',
 '119.shipup',
 '121.dialer',
 '123.almanahe',
 '124.mediafinder',
 '127.dalexis',
 '128.conduit',
 '129.dealply',
 '131.qukart',
 '132.fareit',
 '133.installiq',
 '137.vbna',
 '139.softonic',
 '140.bjlog',
 '141.peed',
 '144.cosmu',
 '151.brresmon',
 '152.navipromo',
 '156.hybris',
 '158.amonetize',
 '163.klez',
 '165.swisyn',
 '172.adgazelle',
 '176.brontok',
 '182.cridex',
 '187.wenper',
 '19.installcore',
 '28.strictor',
 '30.kdz',
 '31.elkern',
 '34.barys',
 '36.delf',
 '38.firseria',
 '39.sirefef',
 '40.msil',
 '41.ibryte',
 '42.simda',
 '43.tepfer',
 '44.startpage',
 '46.shiz',
 '47.mikey',
 '48.upatre',
 '51.ircbot',
 '53.killav',
 '56.onlinegames',
 '59.installrex',
 '60.mydoom',
 '61.virlock',
 '63.aira',
 '64.hupigon',
 '66.renos',
 '67.mywebsearch',
 '70.lollipop',
 '71.mab

In [664]:
rep_dir = './data/tree-rep-profiles_o2o/normal/3.allaple_0.8/G2082/'
rep = pickle.load(open(rep_dir+'rep.pickle','rb'))
# pr = pickle.load(open(rep_dir+'019a496fee9eff1fc2d428982415aefd100bba5c593d7e5b4dff214c6f68ff34_2960_byterep.pickle','rb'))
# pr
rep

[['RegQueryValue#PR@HKLM@sys_curCtlSet_ctl_sessionManager\\*#PR@SUBK@criticalsectiontimeout#PR@0#PR@12f9b0#Ret#0',
  'RegQueryValue#PR@HKLM@soft_ms_ole\\*#PR@SUBK@rwlockresourcetimeout#PR@0#PR@12f9b4#Ret#P'],
 ['LoadLibrary#PR@USR@malware@ENU#Ret#N',
  'LoadLibrary#PR@USR@malware@EN#Ret#N'],
 ['CreateFile#PR@USR@EXE#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_READ#Ret#0',
  'CopyFile#PR@USR@EXE#PR@TMP@EXE#Ret#0',
  'CreateFile#PR@USR@EXE#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_READ#Ret#0',
  'CopyFile#PR@USR@EXE#PR@TMP@EXE#Ret#0',
  'CreateFile#PR@USR@EXE#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_READ#Ret#0',
  'CopyFile#PR@USR@EXE#PR@TMP@EXE#Ret#0',
  'CreateFile#PR@USR@EXE#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_READ#Ret#0',
  'CopyFile#PR@USR@EXE#PR@TMP@EXE#Ret#0',
  'CreateFile#PR@USR@EXE#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_READ#Ret#0',
  'CopyFile#PR@USR@EXE#PR@TMP@EXE#Ret#0',
  'CreateFile#PR@USR@EXE#PR@GENERIC_READ#PR@OPEN_EXISTING#PR@FILE_SHARE_

In [116]:
bound_accuracy,wrong_tre = inf_evaluate(test_pid_paths,test_emb_api,test_emb)
basic_statistics(bound_accuracy)

5it [00:00,  5.89it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
11it [00:01,  6.50it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
29it [00:04,  6.25it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret 

376it [00:56,  6.57it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
385it [00:58,  6.75it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
393it [00:59,  6.55it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(

Exact Match Accuracy Score: 0.734251968503937 EM#:/all#:/No ind REP#: 373 508 125 Exact Match Accuracy Score (w/o No ind REP): 0.9738903394255874


In [112]:
basic_statistics(bound_accuracy)

,length
mean,9.948047e-01
std,1.627907e-02
mode,0.000000e+00
min,3.652893e-01
q1,1.000000e+00
median,1.000000e+00
q3,1.000000e+00
max,1.000000e+00
iqr,1.332268e-15
outlier,1.000000e+00


### Embedding VIZ
* by tree (in 1 fam)
* by fam (nearest K fam)

by tree

In [132]:
fam_name = '3.allaple'
train_dir =  './data/tree-rep-profiles_o2o/profile_avg/'+fam_name+'*' #avg rep的位置
train_avg_pickles = glob.glob(train_dir)
# test_dir = './data/tree-rep-profiles_o2o/TEST/'+fam_name+'_0.8/'
thr = 0.482

In [133]:
train_avg_emb = []
train_tree_name = ['Tree\tType\tID']
for i,avg_emb_path in enumerate(train_avg_pickles):
    tree_name = avg_emb_path.split('/')[-1].split('_')[1]
    emb = pickle.load(open(avg_emb_path,'rb'))[0]
    train_avg_emb.append(emb)
    train_tree_name.append(tree_name+'\t'+'Train'+'\t'+str(i))
i

391

In [134]:
def avg_target_vector(target_pid_paths,target_emb_api,target_emb,thr):
    no_ind_rep = 0
    tree_vec = []
    target_avg_emb = []
    target_tree_name = []
    print(thr)
    i=392
#     if not os.path.exists(root_dir):
#         os.makedirs(root_dir,exist_ok=True)
    for path,api,emb in tqdm(zip(target_pid_paths,target_emb_api,target_emb)): #tqdm
#         recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
#         hash_id = path.split('/')[-1][:5]
#         pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
            print('No common rep. Should not in a tree:',path)
            continue
        assert api[:length][-1] != 0
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0) 
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768) ##api_e,emb_e
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e]) ##api_e,emb_e
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        target_avg_emb.append(rep_emb_avg)
        target_tree_name.append(recent_tree+'\t'+'Test'+'\t'+str(i))
        i=i+1
#         rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
#         pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    return target_avg_emb,target_tree_name

In [135]:
target_id = []
target_pid_paths = []
for i,path in enumerate(test_pid_paths):
    if fam_name in path:
        target_id.append(i)
        target_pid_paths.append(path)
target_emb_api = test_emb_api[target_id]
target_emb = test_emb[target_id]
assert len(target_id) == len(target_emb_api) == len(target_emb) == len(target_pid_paths)
print(target_emb.shape)

(90, 226, 768)


In [136]:
target_avg_emb,target_tree_name = avg_target_vector(target_pid_paths,target_emb_api,target_emb,thr)
assert len(target_avg_emb) == len(target_tree_name)

0it [00:00, ?it/s]

0.482


/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
90it [00:16,  5.84it/s]

No ind REP#: 38


In [137]:
all_avg_emb = train_avg_emb
all_tree_name = train_tree_name
print(len(all_avg_emb),len(all_tree_name))
all_avg_emb.extend(target_avg_emb)
all_tree_name.extend(target_tree_name)
len(all_avg_emb),len(all_tree_name)

392 393


(444, 445)

In [138]:
all_avg_df = pd.DataFrame(np.array(all_avg_emb))
all_avg_df.to_csv('./data/tree-rep-profiles_o2o/allaple_emb.tsv',sep='\t',index=False,header=False)
all_avg_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,1.151180,-0.033245,1.211030,0.658290,3.207358,2.858004,-0.766123,-2.296953,-3.287895,-3.548134,...,0.009648,-1.807576,-3.492755,-0.407835,2.743432,-0.067468,-1.596290,4.217870,2.252217,-3.262894
1,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
2,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
3,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
4,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
5,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
6,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
7,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
8,0.481901,-1.842623,-2.880415,0.608281,3.443483,1.560303,-0.591524,-1.468204,-2.590485,-3.006182,...,1.344966,-0.177493,-5.816375,-2.626812,0.759314,-1.657463,0.500596,2.200020,1.236318,-0.410837
9,-0.614431,-1.836803,-2.593045,1.575328,3.688454,1.176005,-1.279995,-1.243885,-3.267095,-3.591264,...,1.739043,-0.705363,-2.881520,-1.764409,1.477723,0.472045,-0.591169,2.314564,1.595199,-0.354043


In [139]:
with open ("./data/tree-rep-profiles_o2o/allaple_meta.tsv","w")as fp:
    for line in all_tree_name:
        fp.write(line+"\n")

by fam (all 47's train)

In [388]:
train_dir =  './data/tree-rep-profiles_o2o/profile_avg/'+'*' #avg rep的位置
train_avg_pickles = glob.glob(train_dir)
# test_dir = './data/tree-rep-profiles_o2o/TEST/'+fam_name+'_0.8/'
thr = 0.482

In [389]:
train_avg_emb = []
train_tree_name = ['Family\tTree\tType']
for i,avg_emb_path in enumerate(train_avg_pickles):
    fam_name = avg_emb_path.split('/')[-1].split('_')[0]
    tree_name = avg_emb_path.split('/')[-1].split('_')[1]
    emb = pickle.load(open(avg_emb_path,'rb'))[0]
    train_avg_emb.append(emb)
    train_tree_name.append(fam_name+'\t'+tree_name+'\t'+'Train')
i

3124

In [392]:
assert len(train_avg_emb) + 1 == len(train_tree_name)
fam_avg_df = pd.DataFrame(np.array(train_avg_emb))
fam_avg_df.to_csv('./data/tree-rep-profiles_o2o/47fam_emb.tsv',sep='\t',index=False,header=False)
fam_avg_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.481911,-1.433070,-1.635722,1.679263,3.127853,2.005435,-1.032501,-1.338065,-1.541768,-3.073612,...,1.099642,-2.179858,-4.080650,-1.260161,0.384816,0.859065,-0.550434,2.699001,1.448644,-1.232464
1,1.151180,-0.033245,1.211030,0.658290,3.207358,2.858004,-0.766123,-2.296953,-3.287895,-3.548134,...,0.009648,-1.807576,-3.492755,-0.407835,2.743432,-0.067468,-1.596290,4.217870,2.252217,-3.262894
2,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
3,-0.707518,-1.935717,-2.797262,1.670851,3.781644,1.134658,-1.316523,-1.113827,-3.381308,-3.621888,...,1.799625,-0.588610,-2.908403,-1.878712,1.370275,0.413874,-0.604043,2.297332,1.594553,-0.288157
4,0.519463,-0.922928,0.138235,1.692138,3.751782,2.562369,-1.327235,-2.312651,-1.622840,-2.669979,...,1.638167,-3.357604,-4.104876,-0.348341,1.557325,1.362026,-1.586283,2.804302,2.092132,-1.679596
5,0.213354,-1.375378,-1.006419,1.039247,2.814053,0.985314,-1.652755,-1.769655,-2.803002,-2.672520,...,1.286125,-1.708529,-4.324072,-2.046962,1.585701,-0.597090,-0.354960,1.719882,1.554423,-0.457560
6,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
7,-0.188679,-1.634480,-1.778155,0.911086,3.479414,1.611645,-1.257667,-1.394035,-2.405012,-2.623894,...,1.560179,-1.933544,-3.768912,-1.589818,1.338310,0.142061,-1.151662,2.501948,2.180527,-0.925094
8,0.257902,-1.196313,-0.323242,1.190057,3.556135,2.098753,-1.320948,-1.990033,-1.802201,-2.325113,...,1.949429,-2.648337,-3.950665,-0.651112,1.610570,0.912417,-1.029061,2.687641,1.877151,-1.502330
9,-0.994883,-2.221080,-2.698701,0.920521,3.143756,1.414650,-0.806535,-0.868412,-1.565457,-3.673240,...,0.569579,-1.563029,-4.179900,-1.870185,-0.670016,-0.495346,-1.161627,2.532974,1.525146,-0.363356


In [393]:
with open ("./data/tree-rep-profiles_o2o/47fam_meta.tsv","w")as fp:
    for line in train_tree_name:
        fp.write(line+"\n")

loner topN

In [646]:
fam_target = 'allaple'
train_dir =  './data/tree-rep-profiles_o2o/profile_avg/'+'*' #avg rep的位置
train_avg_pickles = glob.glob(train_dir)
# test_dir = './data/tree-rep-profiles_o2o/TEST/'+fam_name+'_0.8/'
thr = 0.482

In [647]:
train_avg_emb = []
train_tree_name = ['Family\tTree\tType']
for i,avg_emb_path in enumerate(train_avg_pickles):
    fam_name = avg_emb_path.split('/')[-1].split('_')[0].split('.')[1]
    tree_name = fam_name+'_'+avg_emb_path.split('/')[-1].split('_')[1]
    emb = pickle.load(open(avg_emb_path,'rb'))[0]
    train_avg_emb.append(emb)
    train_tree_name.append(fam_name+'\t'+tree_name+'\t'+'Train')
i

3124

In [648]:
def avg_target_vector_fam(target_pid_paths,target_emb_api,target_emb,thr):
    no_ind_rep = 0
    tree_vec = []
    target_avg_emb = []
    target_fam_name = []
    print(thr)
    i=392 #
#     if not os.path.exists(root_dir):
#         os.makedirs(root_dir,exist_ok=True)
    for path,api,emb in tqdm(zip(target_pid_paths,target_emb_api,target_emb)): #tqdm
        recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = recent_fam + '_' + path.split('/')[-2]
#         hash_id = path.split('/')[-1][:5]
#         pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
            print('No common rep. Should not in a tree:',path)
            continue
        assert api[:length][-1] != 0
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0) 
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768) ##api_e,emb_e
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e]) ##api_e,emb_e
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        target_avg_emb.append(rep_emb_avg)
        target_fam_name.append(recent_fam+'\t'+recent_tree+'\t'+'Loner')
#         i=i+1
#         rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
#         pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
    print('No ind REP#:',no_ind_rep)
    return target_avg_emb,target_fam_name

In [649]:
target_id = []
target_pid_paths = []
for i,path in enumerate(exp_pid_paths):
    if (fam_target in path) and (path in topN_path): #topN_path要先跑下面loner
        target_id.append(i)
        target_pid_paths.append(path)
target_emb_api = exp_api[target_id]
target_emb = exp_emb[target_id]
assert len(target_id) == len(target_emb_api) == len(target_emb) == len(target_pid_paths)
print(target_emb.shape)

(115, 226, 768)


In [650]:
target_avg_emb,target_fam_name = avg_target_vector_fam(target_pid_paths,target_emb_api,target_emb,thr)
assert len(target_avg_emb) == len(target_fam_name)

1it [00:00,  5.27it/s]

0.482


115it [00:19,  6.05it/s]

No ind REP#: 0


In [651]:
all_avg_emb = []
all_avg_emb.extend(train_avg_emb)
all_fam_name = []
all_fam_name.extend(train_tree_name)
assert len(all_avg_emb)+1 == len(all_fam_name)
all_avg_emb.extend(target_avg_emb)
all_fam_name.extend(target_fam_name)
assert len(all_avg_emb)+1 == len(all_fam_name)

In [654]:
fam_avg_df = pd.DataFrame(np.array(all_avg_emb))
fam_avg_df.to_csv('./data/tree-rep-profiles_o2o/allapleLoner_fam_emb2.tsv',sep='\t',index=False,header=False)
fam_avg_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.481911,-1.433070,-1.635722,1.679263,3.127853,2.005435,-1.032501,-1.338065,-1.541768,-3.073612,...,1.099642,-2.179858,-4.080650,-1.260161,0.384816,0.859065,-0.550434,2.699001,1.448644,-1.232464
1,1.151180,-0.033245,1.211030,0.658290,3.207358,2.858004,-0.766123,-2.296953,-3.287895,-3.548134,...,0.009648,-1.807576,-3.492755,-0.407835,2.743432,-0.067468,-1.596290,4.217870,2.252217,-3.262894
2,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
3,-0.707518,-1.935717,-2.797262,1.670851,3.781644,1.134658,-1.316523,-1.113827,-3.381308,-3.621888,...,1.799625,-0.588610,-2.908403,-1.878712,1.370275,0.413874,-0.604043,2.297332,1.594553,-0.288157
4,0.519463,-0.922928,0.138235,1.692138,3.751782,2.562369,-1.327235,-2.312651,-1.622840,-2.669979,...,1.638167,-3.357604,-4.104876,-0.348341,1.557325,1.362026,-1.586283,2.804302,2.092132,-1.679596
5,0.213354,-1.375378,-1.006419,1.039247,2.814053,0.985314,-1.652755,-1.769655,-2.803002,-2.672520,...,1.286125,-1.708529,-4.324072,-2.046962,1.585701,-0.597090,-0.354960,1.719882,1.554423,-0.457560
6,-0.562403,-1.824229,-2.525939,1.604070,3.669283,1.207198,-1.244334,-1.222941,-3.246776,-3.559425,...,1.722269,-0.741045,-2.826406,-1.729772,1.501466,0.503164,-0.588141,2.381776,1.587046,-0.378239
7,-0.188679,-1.634480,-1.778155,0.911086,3.479414,1.611645,-1.257667,-1.394035,-2.405012,-2.623894,...,1.560179,-1.933544,-3.768912,-1.589818,1.338310,0.142061,-1.151662,2.501948,2.180527,-0.925094
8,0.257902,-1.196313,-0.323242,1.190057,3.556135,2.098753,-1.320948,-1.990033,-1.802201,-2.325113,...,1.949429,-2.648337,-3.950665,-0.651112,1.610570,0.912417,-1.029061,2.687641,1.877151,-1.502330
9,-0.994883,-2.221080,-2.698701,0.920521,3.143756,1.414650,-0.806535,-0.868412,-1.565457,-3.673240,...,0.569579,-1.563029,-4.179900,-1.870185,-0.670016,-0.495346,-1.161627,2.532974,1.525146,-0.363356


In [655]:
with open ("./data/tree-rep-profiles_o2o/allapleLoner_fam_emb_meta2.tsv","w")as fp:
    for line in all_fam_name:
        fp.write(line+"\n")

## Loner EXP
* 同分遞移
* 同hash、不同pid歸類fam

In [398]:
def statistic_hash_pids(root_dir):
    fam_dir = next(os.walk(root_dir))[1]
    all_pids= []
    all_pid_list = []
    tree_count = 0
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        tree_count += len(tree_dir)
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            hash_list = [f.split('/')[-1].split('_')[0] for f in hl_list]
            pid_list = [f.split('/')[-1] for f in hl_list]
            all_pids.extend(hash_list)
            all_pid_list.extend(pid_list)
#             print(fam,len(hash_list)) #DEBUG
    all_hash = set(all_pids)
    print('Families#:',len(fam_dir),'Samples#:',len(all_hash),',Processes#:',len(set(all_pid_list)),',Trees#:',tree_count)
    return all_hash, list(set(all_pid_list))

In [137]:
temp = statistic_hash_pids('./data/tree-rep-profiles_o2o/EXP/')



100%|██████████| 45/45 [00:00<00:00, 1513.82it/s]

Families#: 45 Samples#: 3328 ,Processes#: 3893 ,Trees#: 45


In [824]:
def topN_index(score,sim_score=0.8): #改:多大相似度以上就拿進來
    '''
    input: cosin sim 分數
    output: topN的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) # 取道小數點地五為四捨五入到第四位
    topN_id = []
    for idx in scores:
        if score[0][idx] < sim_score:
            break
        topN_id.append(idx)
    return topN_id

In [825]:

def loner_evaluate(inf_pid_paths,inf_emb_api,inf_emb,ind_reps_matrix, #ind_reps_matrix
                   ind_rep_fam,thr=thr,debug=True): #ind_rep_fam
    em_up = 0
    no_ind_rep_hash = []
    bound_accuracy = []
    wrong_fam = {}
    wrong_fam_em = {}
    processed_hash = {}
    processed_hash_topN = []
    i=-1
    topN_up = 0
    empty_count = 0
    
    EM_path = []
    topN_path = []
    unk_path = []
    claim_path = []
    sample_topNpredict_num = {}
    no_match_anyone = []
    
    #debug
    em_fam_ori = {}
    topN_fam_ori = {}
#     mismatch_path = [] #改成用減的 (calim_path - EM - topN)
    
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)): #tqdm
        i+=1
        split_flag = False
        recent_fam = path.split('/')[-3] #EX: fakeav_fakealert
        if '_' in recent_fam:
            split_flag = True
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
#             print('Empty profile:',path)
            empty_count += 1
            continue
        try:
            assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
            assert api[:length+1][-1] == 0
        except:
#             print('===Assertion ERR:',i,path)
            continue
        recent_hash = path.split('/')[-1].split('_')[0]
        if recent_hash in processed_hash.keys():
            if processed_hash[recent_hash] == 1:
                pass
        else:
            processed_hash[recent_hash] = 0        
        
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep_hash.append(recent_hash)
                unk_path.append(path)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        score = cosine_similarity(rep_emb_avg,ind_reps_matrix)
        
        scores = score[0].argsort() #相似度小到大
        scores = np.flip(scores) # 相似度大到小
        for idx in scores:
            if ind_rep_fam[idx] == recent_fam:
                bound_accuracy.append(score[0][idx]) #最差都是幾分的時候合併 #by_process
        
        idx_list,temp = EM_index(score) # exact match的id
        idx_list_topN = topN_index(score)
        em_fam = []
        for idx in idx_list:
            em_fam.append(ind_rep_fam[idx])
        em_fam = list(set(em_fam)) # exact match的tree name
        topN_fam = []
        for idx in idx_list_topN:
            topN_fam.append(ind_rep_fam[idx])
        topN_fam = list(set(topN_fam))

        if len(topN_fam)>0:
            try:
#                 print(sample_topNpredict_num[recent_hash],topN_fam,recent_hash)
                sample_topNpredict_num[recent_hash].extend(topN_fam)
            except KeyError:
#                 print(topN_fam)
                sample_topNpredict_num[recent_hash] = topN_fam
        else:
#                     if (topN_fam == []) or (topN_fam is None):
            no_match_anyone.append(path)
        
        flag = False #預設沒分對類別
        fams = []
        for fam in em_fam:
            if fam in recent_fam:# recent_fam: 答案
                processed_hash[recent_hash] = 1
                EM_path.append(path)
                flag = True #分對類
                fams.append(fam)
                if split_flag and debug:
                    em_fam_ori[path] = fams
#                     print(path,fam,recent_fam) #profile path、predict、true
                if not debug:
                    break
        if not flag: #依然沒分對類別
            wrong_fam_em[path]=em_fam #by_process最接近的是誰
        if recent_hash not in processed_hash_topN:
            fams = []
            for fam in topN_fam:
                if fam in recent_fam:
                    topN_up = topN_up + 1
                    topN_path.append(path)
                    flag = True
                    fams.append(fam)
                    if split_flag and debug:
                        topN_fam_ori[path] = fams
#                         print(path,fam,recent_fam)
                    if not debug:
                        break
            processed_hash_topN.append(recent_hash)
            if not flag:
                wrong_fam[path]=topN_fam
        claim_path.append(path)
    em_up = sum(list(processed_hash.values()))
    down = len(processed_hash)
    no_ind_rep = len(list(set(no_ind_rep_hash) - set(processed_hash.keys())))#有問題
#     print('ExactMatchAccuracyScore:',em_up/down,'EM_hash#:/all_hash#:/NoIndREP_sample#:',em_up,down,no_ind_rep,
#          'ExactMatchPrecision(w/o No ind REP):',em_up/(down-no_ind_rep),
#           'TopNAccScore:',topN_up/len(processed_hash_topN),'TopNAccScore(w/o No ind REP):',topN_up/(len(processed_hash_topN)-no_ind_rep),
#           'Empty process#:',empty_count
#          )
    return (bound_accuracy,wrong_fam,processed_hash,topN_up/(len(processed_hash_topN)-no_ind_rep),
           EM_path,topN_path,unk_path,claim_path,sample_topNpredict_num,no_match_anyone,
            em_fam_ori,topN_fam_ori) #真正正確的tree是在幾%相似度才有? 、{[正確 path]=predict list}

In [807]:
if True and True:
    print('k')

k


In [ ]:
thr_li = []
topN_score_li = []
# no_rep_num_li = []
# EMw_score_li = []
# rep_avg_len = []
# profile_avg_len = []
for i in tqdm(range(1,100)): #system: 500 (0.002)
    thr = i/100
    ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg_exp/'+str(thr)+'/' # data set dir
    matrix,tree_ans,fam_ans = prepare_inf_data(ind_rep_dir)
    temp1,temp2,temp3,topN_score = loner_evaluate(exp_pid_paths,exp_api,exp_emb
                                                         ,ind_reps_matrix=matrix,ind_rep_fam=fam_ans,thr=thr)
    thr_li.append(thr)
    topN_score_li.append(topN_score)
#     no_rep_num_li.append(no_rep)
#     EMw_score_li.append(EMw_score)
#     rep_avg_len.append(avg_rep)
#     profile_avg_len.append(avg_profile)
# 2:10:40

In [249]:
loner_df = pd.DataFrame(data={'threshold':thr_li,'topN_acc':topN_score_li})
loner_df.to_excel('data/tree-rep-profiles_o2o/threxp_topN_loner.xlsx',index=False)
loner_df

,threshold,topN_acc
0,0.01,0.933993
1,0.02,0.933753
2,0.03,0.935333
3,0.04,0.932686
4,0.05,0.931231
5,0.06,0.930373
6,0.07,0.929801
7,0.08,0.931326
8,0.09,0.930894
9,0.10,0.931479


In [792]:
# claim_path
unk_path

['./data/tree-rep-profiles_o2o/EXP/parite/G0/010b6bbe8e86e417ec383f3e63f63ad8d874ebbcde2f5e11af308e8671cd6c82_2856.profile',
 './data/tree-rep-profiles_o2o/EXP/parite/G0/10f7cb17e9661e7cfae59e9e526b347771609649d0217c6a353defac1ef47fac_3188.profile',
 './data/tree-rep-profiles_o2o/EXP/parite/G0/89ea287969ad604ff55189d449e09cb2e112679087dc10e4d078ecb43679d53d_3544.profile',
 './data/tree-rep-profiles_o2o/EXP/parite/G0/287e67d693027e8054b7f3babfb2cc600a1ab0b4236f13ecd47546863e36dc1d_3004.profile',
 './data/tree-rep-profiles_o2o/EXP/outbrowse/G0/4c1228f6826459914ab2d51c222c40e5360e2b77b68bd8bfe21909832bd93b1c_2960.profile',
 './data/tree-rep-profiles_o2o/EXP/outbrowse/G0/e8971475d8e59ba4cffbcf9d59c8d8b81d8160038da1529b100fd5ec39774c25_440.profile',
 './data/tree-rep-profiles_o2o/EXP/outbrowse/G0/d0a5e43de28c22a386c0f8c2e61614ef86eec2fc690571263791e94e25f52703_3356.profile',
 './data/tree-rep-profiles_o2o/EXP/outbrowse/G0/95b3dcf92ec54b1fca6f05360bc5da5cd5e1b61ef54e0e28cf24eda5db8ec876_3512

In [248]:
# score_df = pd.DataFrame(data={'threshold':thr_li,'topN_score':topN_score_li})
# score_df.to_excel('data/tree-rep-profiles_o2o/threxp_topN_loner.xlsx',index=False)
# score_df

[0.933993399339934,
 0.9337528983106989,
 0.9353333333333333,
 0.9326858673811118,
 0.9312311304931231,
 0.9303733602421796,
 0.9298008774890314,
 0.9313261163734776,
 0.9308943089430894,
 0.9314789687924017,
 0.9309758585515131,
 0.9318568994889267,
 0.9296207721216262,
 0.9288645690834473,
 0.9277397260273973,
 0.9266117969821673,
 0.9261422191686706,
 0.925671025464556,
 0.926946933149552,
 0.926232333678042,
 0.9251466022766471,
 0.9244306418219461,
 0.9232895646164478,
 0.9227304227304227,
 0.9230769230769231,
 0.9230235783633842,
 0.9257717655220257,
 0.9270833333333334,
 0.9258872651356994,
 0.9261324041811847,
 0.9260551098709452,
 0.9248777078965759,
 0.925848198670864,
 0.9254462723136156,
 0.9247462373118656,
 0.9243697478991597,
 0.9235892043463021,
 0.9236990154711674,
 0.9199294532627866,
 0.9215824796891557,
 0.9213323883770376,
 0.9209500177242113,
 0.9204827831025915,
 0.9188612099644128,
 0.9170818505338079,
 0.9166666666666666,
 0.9125936496610774,
 0.913447782546495

In [826]:
# thr_li = []
# topN_score_li = []
# no_rep_num_li = []
# EMw_score_li = []
# rep_avg_len = []
# profile_avg_len = []
# for i in tqdm(range(1,100)): #system: 500 (0.002)
#     thr = 0.48
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/'#'/'+str(thr)+'/' # data set dir
matrix,tree_ans,fam_ans = prepare_inf_data(ind_rep_dir)
(temp1,loner_misFam_08,temp3,topN_score,
EM_path,topN_path,unk_path,claim_path,
 sample_topNpredict_num,no_match_anyone_path,
em_split_fam_ori,top08_split_fam_ori) = loner_evaluate(exp_pid_paths,exp_api,exp_emb
                                                         ,ind_reps_matrix=matrix,ind_rep_fam=fam_ans,thr=0.482)
#     thr_li.append(thr)
#     topN_score_li.append(topN_score)
#     no_rep_num_li.append(no_rep)
#     EMw_score_li.append(EMw_score)
#     rep_avg_len.append(avg_rep)
#     profile_avg_len.append(avg_profile)

5it [00:00,  6.91it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
32it [00:04,  6.49it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
36it [00:04,  6.14it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret 

394it [00:55,  6.52it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
402it [00:56,  7.04it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
404it [00:56,  7.02it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(

659it [01:31,  7.02it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
672it [01:33,  6.88it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
680it [01:34,  6.09it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(

1034it [02:20,  7.15it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1043it [02:21,  7.13it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1047it [02:22,  6.72it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

1297it [02:56,  6.62it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1299it [02:56,  6.56it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1303it [02:57,  6.88it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

...

2135it [04:48,  7.84it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2137it [04:49,  6.90it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2142it [04:49,  6.37it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

2406it [05:27,  7.09it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2439it [05:32,  6.57it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2442it [05:32,  6.32it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

2849it [06:29,  6.68it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2851it [06:29,  6.95it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2872it [06:32,  6.43it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

3206it [07:17,  6.71it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
3247it [07:23,  6.36it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
3253it [07:24,  6.69it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

3580it [08:12,  6.33it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
3591it [08:14,  6.57it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
3604it [08:16,  8.15it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.ty

還原aliases

In [828]:
len(em_split_fam_ori),len(top08_split_fam_ori),len(top09_split_fam_ori)

(134, 414, 404)

In [845]:
exp_dir = './data/tree-rep-profiles_o2o/EXP/'
processed_path = []
for path,fam_li in em_split_fam_ori.items():
    if len(fam_li) > 1:
        continue
    dest_dir = './data/tree-rep-profiles_o2o/EXP_rev/'+fam_li[0]+'/G0/'
    ori_fam = path.split('/')[4]
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir,exist_ok=True)
    path_hash = path.split('/')[-1].split('_')[0]
    all_path_hash = glob.glob(exp_dir+ori_fam+'/G0/'+path_hash+'*')
    for path_glob in all_path_hash:
        if path_glob not in (claim_path + unk_path):
            continue
        if path_glob in processed_path:
            continue
        profile_name = path_glob.split('/')[-1]
        processed_path.append(path_glob)
        shutil.copy(path_glob,dest_dir+profile_name)

for path,fam_li in top09_split_fam_ori.items():
    if len(fam_li) > 1:
        continue
    dest_dir = './data/tree-rep-profiles_o2o/EXP_rev/'+fam_li[0]+'/G0/'
    ori_fam = path.split('/')[4]
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir,exist_ok=True)
    path_hash = path.split('/')[-1].split('_')[0]
    all_path_hash = glob.glob(exp_dir+ori_fam+'/G0/'+path_hash+'*')
    for path_glob in all_path_hash:
        if path_glob not in (claim_path + unk_path):
            continue
        if path_glob in processed_path:
            continue
        profile_name = path_glob.split('/')[-1]
        processed_path.append(path_glob)
        shutil.copy(path_glob,dest_dir+profile_name)
        
for path,fam_li in top08_split_fam_ori.items():
    if len(fam_li) > 1:
        continue
    dest_dir = './data/tree-rep-profiles_o2o/EXP_rev/'+fam_li[0]+'/G0/'
    ori_fam = path.split('/')[4]
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir,exist_ok=True)
    path_hash = path.split('/')[-1].split('_')[0]
    all_path_hash = glob.glob(exp_dir+ori_fam+'/G0/'+path_hash+'*')
    for path_glob in all_path_hash:
        if path_glob not in (claim_path + unk_path):
            continue        
        if path_glob in processed_path:
            continue
        profile_name = path_glob.split('/')[-1]
        processed_path.append(path_glob)
        shutil.copy(path_glob,dest_dir+profile_name)
processed_path = list(set(processed_path))
len(processed_path)

109

In [847]:
processed_hash = []
for path in processed_path:
    current_hash = path.split('/')[-1].split('_')[0]
    processed_hash.append(current_hash)
current_hash = list(set(processed_hash))
len(current_hash)

95

In [851]:
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
fam_dir = './data/tree-rep-profiles_o2o/EXP/downloa_soft/G0/' #fakeav_fakealert/ downloa_soft/
all_profiles = next(os.walk(fam_dir))[2]
for profile in all_profiles:
    current_hash = profile.split('_')[0]
    if current_hash in processed_hash:
        continue
    if (fam_dir + profile) not in (claim_path + unk_path):
        continue
    if (fam_dir + profile) in processed_path:
        continue
    processed_path.append(fam_dir + profile)
    all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
    final_fam = all_path_un_aliase[0].split('/')[4]
    dest_dir = './data/tree-rep-profiles_o2o/EXP_rev/'+final_fam+'/G0/'
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir,exist_ok=True)    
    shutil.copy(fam_dir + profile,dest_dir+profile)
len(processed_path)
        

511

In [854]:
exp_dir = './data/tree-rep-profiles_o2o/EXP/'
fam_dirs = next(os.walk(exp_dir))[1]
fam_dirs.remove('fakeav_fakealert')
fam_dirs.remove('downloa_soft')
fam_dirs

['parite',
 'ipamor',
 'yantai',
 'outbrowse',
 'megasearch',
 'autoit',
 'hotbar',
 'lydra',
 'directdow',
 'solimba',
 'domaiq',
 'eggnog',
 'zusy',
 'picsys',
 'sirefef',
 'browsefox',
 'zbot',
 'ursu',
 'kazy',
 'sytro',
 'pcclient',
 'shodi',
 'allaple',
 'graftor',
 'katusha',
 'mira',
 'fesber',
 'razy',
 'morstar',
 'expiro',
 'sality',
 'symmi',
 'softpulse',
 'vilsel',
 'bdmj',
 'ramnit',
 'installerex',
 'vobfus',
 'somoto',
 'clickdownload',
 'mplug',
 'medfos',
 'virut']

In [855]:
for fam_dir in fam_dirs:
    in_dir = exp_dir+fam_dir+'/G0/'
    fam_profiles = next(os.walk(in_dir))[2]
    for profile in fam_profiles:
        if (in_dir + profile) not in (claim_path + unk_path):
            continue
        if (in_dir + profile) in processed_path:
            continue
        processed_path.append(in_dir + profile)
        dest_dir = './data/tree-rep-profiles_o2o/EXP_rev/'+fam_dir+'/G0/'
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir,exist_ok=True)
        shutil.copy(in_dir + profile,dest_dir+profile)
len(processed_path)

3511

In [722]:
all_sha = []
for path in loner_misFam_09.keys():
    sha = path.split('/')[6].split('_')[0]
    all_sha.append(sha)
(set(all_sha))

{'00065ddb70e8d6977ac7dec4d0c399f8ef183a9534719f9bc0455bf6c9a85143',
 '000fec66b0e30f664a8d3ca1b21c81042912c34dd15d947dff44d7100b9ac061',
 '0083e2c3dbec388b05420477fce1a99bbdd5209d12011a80eecf7a2eb9da576a',
 '009ff7fe9413123da18038211eaeda84e24a25a0233d1d55b9be19230668f944',
 '00f3c6b376c23c543a75b3ed3cd141acee946ae1af0c5741aa123ac7724b08a9',
 '01a9fc05cde93947f3b2d98f23ffea797c2c3327d5226baae74312c4d695b4e5',
 '01b3aeff0603a00fe1cddc525df841a1332b0151d318cc51c7d4d0cce035c46a',
 '01bd29ad98b45d78760c27b483bffa1b60c05045d5f4198f3fbadf540b8483ab',
 '022bb126f7814a0f3b4a59dc9c9e6e7116c34298adbabf496b0e37d8fe481b1a',
 '028d3e185ca95422b7bd3c940e7390368b558bcbc27b5a9d2403a3c1a6782243',
 '02c0d451be0c1fb3689b3478504d4e48f039683c058fd97b8e53b658ed7b47d9',
 '02dd273c9b15c607329d8cad38888aa96c20712533fbd69572b4d841ba0ef8ff',
 '03d44943943e18cfb9e1e71292d1928d8883c1dc9b29b08e837c1ccf1ef76623',
 '03e1a3b94906fdbdcdcce0fceeb2674d3a646f2098f8c3dd02b4ab136e58e0a5',
 '042369b18eb0064c299559622168f69c

unk analyze
* unk_path是否皆是child? (claim_path)
* unk process是來自哪個fam

In [604]:
all_hash_in_loner = list(set(temp3))
print('loner dataset hash數量:',len(all_hash_in_loner),'processes數量:',len(claim_path)+len(unk_path))

loner dataset hash數量: 3045 processes數量: 3511


In [605]:
match_mismatch_hash = [x.split('/')[-1].split('_')[0] for x in claim_path]
print('Samples# in topN EM Mis:',len(set(match_mismatch_hash)),'pids#',len(match_mismatch_hash))
unk_hash = list(set([x.split('/')[-1].split('_')[0] for x in unk_path]))
print('Samples# of unk:' ,len(unk_hash),'Processes# of unk:',len(unk_path))

Samples# in topN EM Mis: 2796 pids# 3192
Samples# of unk: 309 Processes# of unk: 319


In [480]:
pure_no_rep_sample = []
is_child = []
is_main = []
for path in unk_path:
    hash_id = path.split('/')[-1].split('_')[0]
    pid = int(path.split('/')[-1].split('_')[1].split('.')[0])
    corr_samples = [x for x in claim_path if hash_id in x]
    if corr_samples == []:
        pure_no_rep_sample.append(path)
    else:
        corr_samples_pid = [int(x.split('/')[-1].split('_')[1].split('.')[0]) for x in corr_samples]
        if pid>min(corr_samples_pid):
            is_child.append(path)
        else:
            is_main.append(path)
pure_no_rep_sample_hash = list(set([x.split('/')[-1].split('_')[0] for x in pure_no_rep_sample]))
print('某loner完全沒有任何一個process有REP的process數量:',len(pure_no_rep_sample),'Samples數量:',len(pure_no_rep_sample_hash))
print('有其他process有REP，但有部分process沒REP且屬於child的pid數量是:',len(is_child),'沒REP是main pid的數量:',len(is_main))

某loner完全沒有任何一個process有REP的process數量: 253 Samples數量: 249
有其他process有REP，但有部分process沒REP且屬於child的pid數量是: 44 沒REP是main pid的數量: 22


In [682]:
loner_no_rep_pd = pd.DataFrame(columns=['fam','sha256','pid'])
for i,path in enumerate(pure_no_rep_sample):
    fam = path.split('/')[4].split('_')[0].split('.')[-1]
#     tree = path.split('/')[5]
    sha256 = path.split('/')[6].split('_')[0]
    pid = path.split('/')[-1].split('_')[-1].split('.')[0]
    loner_no_rep_pd.loc[i] = [fam,sha256,pid]
loner_no_rep_pd

,fam,sha256,pid
0,parite,010b6bbe8e86e417ec383f3e63f63ad8d874ebbcde2f5e...,2856
1,parite,10f7cb17e9661e7cfae59e9e526b347771609649d0217c...,3188
2,outbrowse,4c1228f6826459914ab2d51c222c40e5360e2b77b68bd8...,2960
3,outbrowse,d0a5e43de28c22a386c0f8c2e61614ef86eec2fc690571...,3356
4,outbrowse,d97b1ef01d24ed09c1f31b0b872d985fde3a311ee3e10d...,3308
5,outbrowse,a86e1de55e31022face5d740ba19d08cd04038dbe17b29...,3244
6,autoit,79c027b36fb76af9f6b26bc1497b579d99626bb7e357e2...,3368
7,autoit,11d20434aae0fd2f633f78ae1d1c4c6f6aced9ada8e7dc...,3384
8,autoit,0c74e9d3da49847f5cb9ea7f6b837abd73f148b5627209...,3352
9,autoit,ff05b34625edbb7ecf343bb79f3e7f3bf14170c9fc0706...,3276


In [683]:
loner_no_rep_pd.to_excel('./data/tree-rep-profiles_o2o/loner_no_rep_hash.xlsx',index=False)

In [481]:
# claim_path + unk_path = all_path去看每個家族有多少samples(hash)=>dixt: [fam] = #
# pure_no_rep_sample=> [hash] = fam
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
all_path = []
all_path.extend(claim_path)
all_path.extend(unk_path)

processed_sample = []
fam_hash_num = {} #每個family在loner set當中有多少samples
for path in all_path:
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
    if '_' in current_fam:
        all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
        current_fam = all_path_un_aliase[0].split('/')[4]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_num[current_fam] = fam_hash_num[current_fam] + 1 
    except KeyError:
        fam_hash_num[current_fam] = 1
fam_hash_num ,len(fam_hash_num) , sum(list(fam_hash_num.values()))

({'parite': 52,
  'ipamor': 11,
  'yantai': 11,
  'outbrowse': 43,
  'megasearch': 21,
  'autoit': 59,
  'hotbar': 71,
  'lydra': 10,
  'directdow': 25,
  'solimba': 112,
  'domaiq': 76,
  'eggnog': 15,
  'zusy': 121,
  'picsys': 16,
  'sirefef': 23,
  'browsefox': 62,
  'zbot': 167,
  'ursu': 122,
  'kazy': 200,
  'sytro': 52,
  'pcclient': 11,
  'shodi': 28,
  'allaple': 138,
  'graftor': 106,
  'katusha': 1,
  'mira': 35,
  'fesber': 20,
  'razy': 68,
  'morstar': 34,
  'expiro': 117,
  'fakealert': 73,
  'fakeav': 167,
  'sality': 88,
  'symmi': 138,
  'softpulse': 48,
  'vilsel': 22,
  'bdmj': 11,
  'ramnit': 73,
  'installerex': 18,
  'vobfus': 79,
  'somoto': 31,
  'clickdownload': 17,
  'downloa': 93,
  'soft': 119,
  'mplug': 60,
  'medfos': 19,
  'virut': 162},
 47,
 3045)

In [603]:
fam_unkhash_num_pd = pd.DataFrame([fam_hash_num],index=['fam_unkhash_num']).T
# fam_unkhash_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_unkhash_num.xlsx',index=True)
fam_unkhash_num_pd

,fam_unkhash_num
allaple,138
autoit,59
bdmj,11
browsefox,62
clickdownload,17
directdow,25
domaiq,76
downloa,93
eggnog,15
expiro,117


In [482]:
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
processed_sample = []
fam_unkhash_num = {} #每個family在loner set當中有多少unk samples
for path in pure_no_rep_sample:
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
    if '_' in current_fam:
        all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
        current_fam = all_path_un_aliase[0].split('/')[4]    
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_unkhash_num[current_fam] = fam_unkhash_num[current_fam] + 1 
    except KeyError:
        fam_unkhash_num[current_fam] = 1
fam_unkhash_num ,len(fam_unkhash_num)

({'parite': 2,
  'outbrowse': 4,
  'autoit': 5,
  'hotbar': 6,
  'directdow': 3,
  'solimba': 11,
  'domaiq': 7,
  'eggnog': 1,
  'zusy': 14,
  'picsys': 1,
  'sirefef': 1,
  'browsefox': 1,
  'zbot': 18,
  'ursu': 12,
  'kazy': 22,
  'sytro': 2,
  'pcclient': 2,
  'shodi': 2,
  'allaple': 14,
  'graftor': 6,
  'mira': 3,
  'fesber': 3,
  'razy': 10,
  'morstar': 3,
  'expiro': 8,
  'fakealert': 8,
  'fakeav': 15,
  'sality': 3,
  'symmi': 13,
  'softpulse': 2,
  'vilsel': 3,
  'ramnit': 2,
  'installerex': 2,
  'vobfus': 7,
  'somoto': 2,
  'clickdownload': 3,
  'soft': 10,
  'downloa': 4,
  'mplug': 3,
  'virut': 11},
 40)

In [598]:
fam_unkhash_num_pd = pd.DataFrame([fam_unkhash_num],index=['fam_unkhash_num']).T
fam_unkhash_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_unkhash_num.xlsx',index=True)
fam_unkhash_num_pd

,fam_unkhash_num
allaple,14
autoit,5
browsefox,1
clickdownload,3
directdow,3
domaiq,7
downloa,4
eggnog,1
expiro,8
fakealert,8


In [576]:
# draw all family unk rate
unk_rate = {}
for fam,all_num in fam_hash_num.items():
    try:
        up = fam_unkhash_num[fam]
    except KeyError:
        up=0
    unk_rate[fam] = up/all_num
unk_pd = pd.DataFrame([unk_rate],index=['Unknown_rate']).T
unk_pd.to_excel('data/tree-rep-profiles_o2o/FamilyUNK_rate_loner.xlsx',index=True)
unk_pd

,Unknown_rate
allaple,0.101449
autoit,0.084746
bdmj,0.000000
browsefox,0.016129
clickdownload,0.176471
directdow,0.120000
domaiq,0.092105
downloa,0.043011
eggnog,0.066667
expiro,0.068376


In [497]:
unk_rate

{'parite': 0.038461538461538464,
 'ipamor': 0.0,
 'yantai': 0.0,
 'outbrowse': 0.09302325581395349,
 'megasearch': 0.0,
 'autoit': 0.0847457627118644,
 'hotbar': 0.08450704225352113,
 'lydra': 0.0,
 'directdow': 0.12,
 'solimba': 0.09821428571428571,
 'domaiq': 0.09210526315789473,
 'eggnog': 0.06666666666666667,
 'zusy': 0.11570247933884298,
 'picsys': 0.0625,
 'sirefef': 0.043478260869565216,
 'browsefox': 0.016129032258064516,
 'zbot': 0.10778443113772455,
 'ursu': 0.09836065573770492,
 'kazy': 0.11,
 'sytro': 0.038461538461538464,
 'pcclient': 0.18181818181818182,
 'shodi': 0.07142857142857142,
 'allaple': 0.10144927536231885,
 'graftor': 0.05660377358490566,
 'katusha': 0.0,
 'mira': 0.08571428571428572,
 'fesber': 0.15,
 'razy': 0.14705882352941177,
 'morstar': 0.08823529411764706,
 'expiro': 0.06837606837606838,
 'fakealert': 0.1095890410958904,
 'fakeav': 0.08982035928143713,
 'sality': 0.03409090909090909,
 'symmi': 0.09420289855072464,
 'softpulse': 0.041666666666666664,
 'vi

Mismatch analyze

In [862]:
EM_hash = [x.split('/')[-1].split('_')[0] for x in EM_path]
topN_hash = [x.split('/')[-1].split('_')[0] for x in topN_path]
mismatch_hash = list(set(match_mismatch_hash) - set(topN_hash) -set(EM_hash))
print('mismatch sample#:',len(mismatch_hash))
exp_dir = './data/tree-rep-profiles_o2o/EXP/'
mismatch_path = []
for hash_m in mismatch_hash:
    sample_pids = glob.glob(exp_dir+'*/*/'+hash_m+'*')
    mismatch_path.extend(sample_pids)
# mismatch_path = list(set(claim_path) - set(EM_path) - set(topN_path))
# mismatch_hash = [x.]
# .split('/')[-1].split('_')[0]
print('mismatch process#:',len(mismatch_path))

mismatch sample#: 145
mismatch process#: 176


In [723]:
len(mismatch_path)

298

In [725]:
mismatch_path

['./data/tree-rep-profiles_o2o/EXP/domaiq/G0/12b35d848640bf60b4986d670654d29c46c97d0f466ad72f8ecede727b332ef6_3224.profile',
 './data/tree-rep-profiles_o2o/EXP/domaiq/G0/9b2872cee6bc8c1e75de775ef7796e2e0efa8aae97411ccc151e99b4df0bb759_3268.profile',
 './data/tree-rep-profiles_o2o/EXP/symmi/G0/3eb7702c7d88bf4eacbd929ae2ed38eeee3a8775b3ff1e5524e97ed88d7f8e9c_2920.profile',
 './data/tree-rep-profiles_o2o/EXP/symmi/G0/3eb7702c7d88bf4eacbd929ae2ed38eeee3a8775b3ff1e5524e97ed88d7f8e9c_3288.profile',
 './data/tree-rep-profiles_o2o/EXP/solimba/G0/47227a80ecb4d5dd562c0702bdf99589c12445f82ca8078b39ca0ff9db661ddf_3292.profile',
 './data/tree-rep-profiles_o2o/EXP/sality/G0/1dbf6233a5e4ecafa7aac6e4c0feb733c72af2fa152d1625751d4bd7bef49117_3220.profile',
 './data/tree-rep-profiles_o2o/EXP/expiro/G0/08a7e8d894730b50b41b233ebcf5e6f569b58c4965f82e69541fb2e3a43f2862_3244.profile',
 './data/tree-rep-profiles_o2o/EXP/sirefef/G0/0a8a1f1edd5456c288dabb2e80b382e54661ec2136b337fc980f00fe271e8b76_3400.profile',


202


,ori_fam,sha256,predict_fam
0,parite,6b1fa2ec3fa6a8059fab46c84e8c5f0f56f65dffffe16b...,[virut]
1,parite,02dd273c9b15c607329d8cad38888aa96c20712533fbd6...,"[mplug, megasearch, fakeav]"
3,yantai,7f632b358594b0d5b1307f45770b8511bb4a8b360192a3...,[virut]
4,outbrowse,d9b6f82f81be335dc4437ebdcf98e2d154608db5a12355...,"[fesber, ursu, zbot, kazy, directdow, parite, ..."
5,outbrowse,0c971c07f8562be15f588459fa14e4fc8aa8eb5bd6fa8c...,"[solimba, directdow, symmi, soft, browsefox, p..."
6,megasearch,12c319744ce4ed98841f037951eb2cce4ed00246796f59...,"[solimba, outbrowse, softpulse, mira, directdo..."
7,megasearch,16da9f5e17d330e569416a0dcbb4fac40ebcfc299f19da...,"[solimba, outbrowse, softpulse, mira, directdo..."
8,megasearch,6e75f8d7e8a9dbd79d6a34fa0c2dc043631a3334e789a8...,"[solimba, outbrowse, softpulse, mira, directdo..."
9,megasearch,10dc3e3b9d9644f15f5c96be62064e0eee80f53870c648...,"[solimba, outbrowse, softpulse, mira, directdo..."
10,megasearch,06cea4c7c21782db93c86c24235b0e6947ffb43e92fb41...,"[solimba, outbrowse, softpulse, mira, directdo..."


In [784]:
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
# all_path = []
# all_path.extend(claim_path)
# all_path.extend(unk_path)

processed_sample = []
fam_hash_woUNK_num = {} #每個family在loner set當中有多少samples是具有REP的
for path in claim_path:
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
    if '_' in current_fam: # un aliase
        all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
        current_fam = all_path_un_aliase[0].split('/')[4]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_woUNK_num[current_fam] = fam_hash_woUNK_num[current_fam] + 1 
    except KeyError:
        fam_hash_woUNK_num[current_fam] = 1
fam_hash_woUNK_num ,len(fam_hash_woUNK_num) , sum(list(fam_hash_woUNK_num.values()))

({'parite': 50,
  'ipamor': 11,
  'yantai': 11,
  'outbrowse': 39,
  'megasearch': 21,
  'autoit': 54,
  'hotbar': 65,
  'lydra': 10,
  'directdow': 22,
  'solimba': 101,
  'domaiq': 69,
  'eggnog': 14,
  'zusy': 107,
  'picsys': 15,
  'sirefef': 22,
  'browsefox': 61,
  'zbot': 149,
  'ursu': 110,
  'kazy': 178,
  'sytro': 50,
  'pcclient': 9,
  'shodi': 26,
  'allaple': 124,
  'graftor': 100,
  'katusha': 1,
  'mira': 32,
  'fesber': 17,
  'razy': 58,
  'morstar': 31,
  'expiro': 109,
  'fakealert': 65,
  'fakeav': 152,
  'sality': 85,
  'symmi': 125,
  'softpulse': 46,
  'vilsel': 19,
  'bdmj': 11,
  'ramnit': 71,
  'installerex': 16,
  'vobfus': 72,
  'somoto': 29,
  'clickdownload': 14,
  'downloa': 89,
  'soft': 109,
  'mplug': 57,
  'medfos': 19,
  'virut': 151},
 47,
 2796)

In [785]:
fam_hash_woUNK_mis_num_pd = pd.DataFrame([fam_hash_woUNK_num],index=['fam_hash_woUNK_mis_num']).T
# fam_hash_woUNK_mis_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_hash_woUNK_mis_num.xlsx',index=True)
fam_hash_woUNK_mis_num_pd

,fam_hash_woUNK_mis_num
allaple,124
autoit,54
bdmj,11
browsefox,61
clickdownload,14
directdow,22
domaiq,69
downloa,89
eggnog,14
expiro,109


In [787]:
fam_hash_woUNK_mis_num_pd.to_excel('./data/tree-rep-profiles_o2o/loner_fam_dist.xlsx')

In [743]:
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
# all_path = []
# all_path.extend(claim_path)
# all_path.extend(unk_path)
loner09_mismatch_fam_pd = pd.DataFrame(columns=['ori_fam','sha256'])
processed_sample = []
fam_hash_woUNK_mis_num = {} #每個family有多少個mismatch sample
for i,path in enumerate(mismatch_path):
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
#     pred = loner_misFam_09[path]
    if current_hash in no_match_sample:
        continue    
    if '_' in current_fam: # un aliase
        all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
        current_fam = all_path_un_aliase[0].split('/')[4]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_woUNK_mis_num[current_fam] = fam_hash_woUNK_mis_num[current_fam] + 1 
    except KeyError:
        fam_hash_woUNK_mis_num[current_fam] = 1
    loner09_mismatch_fam_pd.loc[i] = [current_fam,current_hash]
fam_hash_woUNK_mis_num ,len(fam_hash_woUNK_mis_num) , sum(list(fam_hash_woUNK_mis_num.values()))

({'domaiq': 4,
  'symmi': 1,
  'solimba': 5,
  'sality': 8,
  'expiro': 3,
  'sirefef': 22,
  'mplug': 56,
  'morstar': 4,
  'hotbar': 2,
  'graftor': 10,
  'fakeav': 3,
  'zbot': 3,
  'kazy': 2,
  'sytro': 4,
  'picsys': 2,
  'eggnog': 5,
  'megasearch': 19,
  'vobfus': 1,
  'allaple': 7,
  'lydra': 3,
  'mira': 1,
  'ramnit': 5,
  'downloa': 1,
  'parite': 2,
  'installerex': 4,
  'outbrowse': 2,
  'virut': 5,
  'autoit': 2,
  'ursu': 4,
  'somoto': 2,
  'razy': 2,
  'shodi': 1,
  'browsefox': 2,
  'yantai': 1,
  'bdmj': 1,
  'vilsel': 1,
  'fakealert': 1,
  'softpulse': 1},
 38,
 202)

In [750]:
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
# all_path = []
# all_path.extend(claim_path)
# all_path.extend(unk_path)
loner08_mismatch_fam_pd = pd.DataFrame(columns=['ori_fam','sha256'])
processed_sample = []
fam_hash_woUNK_mis_num = {} #每個family有多少個mismatch sample
for i,path in enumerate(mismatch_path):
    current_hash = path.split('/')[-1].split('_')[0]
    current_fam = path.split('/')[4]
#     pred = loner_misFam_09[path]
    if current_hash in no_match_sample:
        continue    
    if '_' in current_fam: # un aliase
        all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
        current_fam = all_path_un_aliase[0].split('/')[4]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)
    try:
        fam_hash_woUNK_mis_num[current_fam] = fam_hash_woUNK_mis_num[current_fam] + 1 
    except KeyError:
        fam_hash_woUNK_mis_num[current_fam] = 1
    loner08_mismatch_fam_pd.loc[i] = [current_fam,current_hash]
fam_hash_woUNK_mis_num ,len(fam_hash_woUNK_mis_num) , sum(list(fam_hash_woUNK_mis_num.values()))

({'domaiq': 2,
  'solimba': 2,
  'sality': 3,
  'sirefef': 22,
  'mplug': 56,
  'expiro': 3,
  'graftor': 2,
  'sytro': 3,
  'megasearch': 20,
  'ramnit': 3,
  'allaple': 3,
  'downloa': 1,
  'parite': 2,
  'ipamor': 1,
  'autoit': 1,
  'razy': 1,
  'zbot': 4,
  'virut': 4,
  'shodi': 1,
  'yantai': 1,
  'zusy': 1,
  'morstar': 1,
  'vilsel': 1,
  'somoto': 1,
  'softpulse': 1},
 25,
 140)

In [863]:
loner08_mismatch_fam_pd

,ori_fam,sha256
0,domaiq,12b35d848640bf60b4986d670654d29c46c97d0f466ad7...
1,solimba,47227a80ecb4d5dd562c0702bdf99589c12445f82ca807...
2,sality,1dbf6233a5e4ecafa7aac6e4c0feb733c72af2fa152d16...
3,sirefef,0a8a1f1edd5456c288dabb2e80b382e54661ec2136b337...
4,sirefef,671b800461a80430532a77f3799f2a9ee66e99ec8b41bc...
6,mplug,ef2dd909f9d313ffb02a9ee4a4f00b7889711e4ba5b3ac...
8,sirefef,0e077f22906f9c70fa6ca17f6fc04cc28b41fcf813b487...
9,expiro,4e86096facf543bb8088ae0a8900a68fa5450681426925...
11,graftor,233dd2f1a835e371f9a070ca1ce23c32a4e2ec180607aa...
12,mplug,fb1f81fce482bc97d3166a373747cf4cba07eedb2b0afc...


In [751]:
loner08_mismatch_fam_pd.to_excel('./data/tree-rep-profiles_o2o/loner08_mismsatch_fam.xlsx',index=False)

In [744]:
loner09_mismatch_fam_pd.to_excel('./data/tree-rep-profiles_o2o/loner09_mismsatch_fam2.xlsx',index=False)

In [740]:
# loner09_mismatch_fam_pd = pd.DataFrame(columns=['ori_fam','sha256','predict_fam'])
# all_hash = []
# for i , (path,pred) in enumerate(loner_misFam_09.items()):
#     fam = path.split('/')[4].split('_')[0].split('.')[-1]
#     tree = path.split('/')[5]
#     sha256 = path.split('/')[6].split('_')[0]
#     pid = path.split('/')[-1].split('_')[-1].split('.')[0]
#     if sha256 in no_match_sample:
#         continue
# #     if fam == 'sirefef': # train沒有
# #         continue
# #     if path not in mismatch_path:
# #         continue
#     all_hash.append(sha256)
#     loner09_mismatch_fam_pd.loc[i] = [fam,sha256,pred]
# print(len(set(all_hash)))
# loner09_mismatch_fam_pd

202


,ori_fam,sha256,predict_fam
0,parite,6b1fa2ec3fa6a8059fab46c84e8c5f0f56f65dffffe16b...,[virut]
1,parite,02dd273c9b15c607329d8cad38888aa96c20712533fbd6...,"[mplug, megasearch, fakeav]"
3,yantai,7f632b358594b0d5b1307f45770b8511bb4a8b360192a3...,[virut]
4,outbrowse,d9b6f82f81be335dc4437ebdcf98e2d154608db5a12355...,"[fesber, ursu, zbot, kazy, directdow, parite, ..."
5,outbrowse,0c971c07f8562be15f588459fa14e4fc8aa8eb5bd6fa8c...,"[solimba, directdow, symmi, soft, browsefox, p..."
6,megasearch,12c319744ce4ed98841f037951eb2cce4ed00246796f59...,"[solimba, outbrowse, softpulse, mira, directdo..."
7,megasearch,16da9f5e17d330e569416a0dcbb4fac40ebcfc299f19da...,"[solimba, outbrowse, softpulse, mira, directdo..."
8,megasearch,6e75f8d7e8a9dbd79d6a34fa0c2dc043631a3334e789a8...,"[solimba, outbrowse, softpulse, mira, directdo..."
9,megasearch,10dc3e3b9d9644f15f5c96be62064e0eee80f53870c648...,"[solimba, outbrowse, softpulse, mira, directdo..."
10,megasearch,06cea4c7c21782db93c86c24235b0e6947ffb43e92fb41...,"[solimba, outbrowse, softpulse, mira, directdo..."


In [736]:
fam_hash_woUNK_mis_num

{'domaiq': 6,
 'symmi': 1,
 'solimba': 5,
 'sality': 12,
 'expiro': 7,
 'sirefef': 22,
 'mplug': 56,
 'morstar': 5,
 'virut': 9,
 'hotbar': 3,
 'graftor': 11,
 'fakeav': 5,
 'zbot': 6,
 'kazy': 4,
 'sytro': 6,
 'megasearch': 21,
 'picsys': 2,
 'eggnog': 5,
 'vobfus': 1,
 'allaple': 9,
 'ursu': 6,
 'lydra': 3,
 'mira': 1,
 'ramnit': 6,
 'downloa': 3,
 'parite': 2,
 'installerex': 4,
 'ipamor': 1,
 'outbrowse': 2,
 'autoit': 2,
 'somoto': 2,
 'razy': 2,
 'browsefox': 3,
 'shodi': 1,
 'yantai': 1,
 'bdmj': 1,
 'zusy': 1,
 'vilsel': 1,
 'fakealert': 2,
 'softpulse': 1}

In [610]:
fam_hash_woUNK_mis_num_pd = pd.DataFrame([fam_hash_woUNK_mis_num],index=['fam_hash_woUNK_mis_num']).T
fam_hash_woUNK_mis_num_pd.to_excel('data/tree-rep-profiles_o2o/fam_hash_woUNK_mis_num09.xlsx',index=True)
fam_hash_woUNK_mis_num_pd

,fam_hash_woUNK_mis_num
allaple,9
autoit,2
bdmj,1
browsefox,3
domaiq,6
downloa,3
eggnog,5
expiro,7
fakealert,2
fakeav,5


In [571]:
# draw all family unk rate
match_rate = {}
for fam,all_num in fam_hash_woUNK_num.items():
    try:
        up = fam_hash_woUNK_mis_num[fam]
    except KeyError:
        up=0
    match_rate[fam] = 1 - up/all_num
acc_pd = pd.DataFrame([match_rate],index=['Family_Match_rate']).T
acc_pd.to_excel('data/tree-rep-profiles_o2o/FamilyMatch08_rate_loner.xlsx',index=True)
acc_pd

,Family_Match_rate
allaple,0.975806
autoit,0.981481
bdmj,1.000000
browsefox,1.000000
clickdownload,1.000000
directdow,1.000000
domaiq,0.971014
downloa,0.988764
eggnog,1.000000
expiro,0.972477


In [749]:
# all predicted family below  threshold (sample)
print(len(no_match_anyone_path)) #PID
no_match_anyone_path_hash = list(set([x.split('/')[-1].split('_')[0] for x in no_match_anyone_path]))
no_match_sample = list(set(no_match_anyone_path_hash) - set(sample_topNpredict_num.keys()))
print('有多少個sample最近的family的相似度都低於topN thr:',len(no_match_sample))
fam_no_match = []
processed_sample = []
un_aliase_path = './data/tree-rep-profiles_o2o/EXP2/'
for path in no_match_anyone_path:  
    current_hash = path.split('/')[-1].split('_')[0]
    if current_hash in processed_sample:
        continue
    else:
        processed_sample.append(current_hash)  
    current_fam = path.split('/')[4]
    if '_' in current_fam: # un aliase
        all_path_un_aliase = glob.glob(un_aliase_path+'*/*/'+current_hash+'*')
        current_fam = all_path_un_aliase[0].split('/')[4]    
    if current_hash in no_match_sample:
        fam_no_match.append(current_fam)
print(len(fam_no_match))
fam_no_match_stat = dict(Counter(fam_no_match))
fam_no_match_stat

6
有多少個sample最近的family的相似度都低於topN thr: 5
5


{'ursu': 1, 'sality': 2, 'ramnit': 1, 'virut': 1}

In [573]:
hash_topNpredict_num = []
for key, li in sample_topNpredict_num.items():
    hash_topNpredict_num.append(len(set(li)))
print(len(hash_topNpredict_num))
basic_statistics(hash_topNpredict_num)


2791


,length
mean,43.196704
std,5.155101
mode,44.000000
min,1.000000
q1,44.000000
median,44.000000
q3,44.000000
max,46.000000
iqr,0.000000
outlier,44.000000


In [574]:
sample_topNpredict_num_df = dict(Counter(hash_topNpredict_num))
sample_topNpredict_num_df = pd.DataFrame([sample_topNpredict_num_df],index=['topN_fam_predict_num']).T
sample_topNpredict_num_df.to_excel('data/tree-rep-profiles_o2o/topN08_predictFam_num_loner.xlsx',index=True)
sample_topNpredict_num_df

,topN_fam_predict_num
1,4
2,1
3,18
5,8
6,4
9,1
12,4
13,1
16,5
18,3


In [756]:
sample_topNpredict_num_df.topN_fam_predict_num.sum()

2791

In [780]:
sample_belongFam09_pd = pd.DataFrame(columns=['sha256','predict_fam'])
all_values_bySample = []
for i,(key, li) in enumerate(sample_topNpredict_num.items()):
    value = list(set(li))
    all_values_bySample.extend(value)
    sample_belongFam09_pd.loc[i] = [key,value]
sample_belongFam09_pd

,sha256,predict_fam
0,f2f311d2f3983ce15491aef2d50e036e62681473e85af5...,"[solimba, outbrowse, softpulse, mira, directdo..."
1,015214dd9631fa340a399498dd9aeb5072dddecc61adad...,"[solimba, outbrowse, softpulse, mira, directdo..."
2,efe6296790af8021ca461178097090086ee520a5500f38...,"[solimba, outbrowse, mira, directdow, symmi, s..."
3,9cc82ecd2098eeb48ea44132a35228484c1cafa9e0f4cf...,"[solimba, outbrowse, softpulse, mira, directdo..."
4,6b1fa2ec3fa6a8059fab46c84e8c5f0f56f65dffffe16b...,[virut]
5,d9830b4ebde85dceab19763d05e1058725d2a774ea3dcf...,"[solimba, outbrowse, softpulse, mira, directdo..."
6,62727023b97272569958f1399d9da3e7ff302eb1eb89e0...,"[solimba, outbrowse, softpulse, mira, directdo..."
7,08a8646097169f25f8c97c18e1094f12677b7b42b1e165...,"[solimba, outbrowse, softpulse, mira, directdo..."
8,59e4eb2df2128208c56ccdda79d8b7e56a5f83cb953273...,"[solimba, outbrowse, softpulse, mira, directdo..."
9,89ea287969ad604ff55189d449e09cb2e112679087dc10...,"[solimba, outbrowse, softpulse, mira, directdo..."


In [781]:
fam_loner_sampleNum_df = dict(Counter(all_values_bySample))
fam_loner_sampleNum_df = pd.DataFrame([fam_loner_sampleNum_df],index=['loner_sampples_num']).T
fam_loner_sampleNum_df

,loner_sampples_num
allaple,2659
autoit,2616
bdmj,2623
browsefox,2678
clickdownload,2639
directdow,2698
domaiq,2689
downloa,2663
eggnog,2105
expiro,2653


In [782]:
sample_belongFam09_pd.to_excel('./data/tree-rep-profiles_o2o/sample_belongFam09.xlsx',index=False)
fam_loner_sampleNum_df.to_excel('./data/tree-rep-profiles_o2o/fam_loner_sampleNum09.xlsx')

train每個family有多少比例有REP

In [584]:
train_pid_fam_num = {}
for path in train_pid_paths:
    fam_name = path.split('/')[4].split('_')[0].split('.')[-1]
    try:
        train_pid_fam_num[fam_name] = train_pid_fam_num[fam_name] + 1
    except KeyError:
        train_pid_fam_num[fam_name] = 1
train_pid_fam_num , sum(list(train_pid_fam_num.values()))
# train_pid_fam_num_pd = pd.DataFrame([train_pid_fam_num],index=['Family_pid_']).T
# train_pid_fam_num_pd.to_excel('data/tree-rep-profiles_o2o/FamilyMatch08_rate_loner.xlsx',index=True)
# train_pid_fam_num_pd

({'domaiq': 49,
  'ipamor': 21,
  'lydra': 9,
  'zusy': 38,
  'zbot': 253,
  'autoit': 15,
  'vilsel': 117,
  'browsefox': 42,
  'ramnit': 19,
  'sality': 9,
  'sytro': 133,
  'medfos': 9,
  'virut': 139,
  'downloa': 9,
  'ursu': 257,
  'expiro': 63,
  'outbrowse': 51,
  'vobfus': 85,
  'graftor': 22,
  'kazy': 173,
  'solimba': 207,
  'megasearch': 13,
  'hotbar': 60,
  'bdmj': 57,
  'parite': 11,
  'fakeav': 869,
  'somoto': 48,
  'fakealert': 135,
  'directdow': 85,
  'sirefef': 10,
  'shodi': 9,
  'softpulse': 20,
  'katusha': 9,
  'soft': 12,
  'pcclient': 14,
  'picsys': 37,
  'fesber': 61,
  'razy': 31,
  'yantai': 21,
  'symmi': 49,
  'mira': 34,
  'eggnog': 36,
  'mplug': 12,
  'allaple': 719,
  'morstar': 9,
  'clickdownload': 14,
  'installerex': 9},
 4104)

In [585]:
avg_dir = './data/tree-rep-profiles_o2o/profile_avg/'
train_have_rep = next(os.walk(avg_dir))[2]
train_pid_fam_haveREPnum = {}
for pkl_name in train_have_rep:
    fam = pkl_name.split('_')[0].split('.')[-1]
    try:
        train_pid_fam_haveREPnum[fam] = train_pid_fam_haveREPnum[fam] + 1
    except KeyError:
        train_pid_fam_haveREPnum[fam] = 1
train_pid_fam_haveREPnum

{'vilsel': 94,
 'allaple': 392,
 'ipamor': 21,
 'vobfus': 75,
 'zbot': 225,
 'mira': 34,
 'kazy': 133,
 'somoto': 37,
 'razy': 23,
 'ursu': 257,
 'bdmj': 57,
 'fesber': 61,
 'sytro': 127,
 'fakeav': 551,
 'picsys': 29,
 'solimba': 196,
 'outbrowse': 41,
 'virut': 109,
 'eggnog': 36,
 'fakealert': 72,
 'expiro': 55,
 'symmi': 37,
 'hotbar': 60,
 'ramnit': 19,
 'katusha': 9,
 'sality': 9,
 'pcclient': 14,
 'yantai': 21,
 'domaiq': 36,
 'softpulse': 8,
 'zusy': 25,
 'directdow': 85,
 'soft': 12,
 'megasearch': 13,
 'browsefox': 34,
 'clickdownload': 14,
 'graftor': 12,
 'medfos': 9,
 'parite': 11,
 'installerex': 9,
 'shodi': 9,
 'lydra': 9,
 'morstar': 9,
 'autoit': 15,
 'mplug': 12,
 'downloa': 9}

In [587]:
haveREP_rate = {}
for fam,all_num in train_pid_fam_num.items():
    try:
        up = train_pid_fam_haveREPnum[fam]
    except KeyError:
        up=0
    haveREP_rate[fam] =  up/all_num
haveREP_pd = pd.DataFrame([haveREP_rate],index=['Family_Train_haveREP_ratio']).T
haveREP_pd.to_excel('data/tree-rep-profiles_o2o/Family_Train_haveREP_pid_ratio.xlsx',index=True)
haveREP_pd

,Family_Train_haveREP_ratio
allaple,0.545202
autoit,1.000000
bdmj,1.000000
browsefox,0.809524
clickdownload,1.000000
directdow,1.000000
domaiq,0.734694
downloa,1.000000
eggnog,1.000000
expiro,0.873016


In [593]:
haverepnum = sum(list(train_pid_fam_haveREPnum.values()))
cir_cookie = {}
for k,v in train_pid_fam_haveREPnum.items():
    cir_cookie[k] = v/haverepnum
cir_cookie_pd = pd.DataFrame([cir_cookie],index=['Family_Train_inner_ratio']).T
cir_cookie_pd.to_excel('data/tree-rep-profiles_o2o/Family_Train_allpid_ratio.xlsx',index=True)
cir_cookie_pd

,Family_Train_inner_ratio
allaple,0.12544
autoit,0.00480
bdmj,0.01824
browsefox,0.01088
clickdownload,0.00448
directdow,0.02720
domaiq,0.01152
downloa,0.00288
eggnog,0.01152
expiro,0.01760


In [243]:
topN_score

0.9130901287553648

In [156]:
temp = loner_evaluate(exp_pid_paths,exp_api,exp_emb)


0it [00:00, ?it/s]
1it [00:00,  3.97it/s]
2it [00:00,  4.52it/s]
3it [00:00,  5.05it/s]
5it [00:00,  6.26it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/e86fcc08eb8dec19fdac33f2b21e8a55bdf677c03d51fc438373187788c477e4_3748.profile


/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

7it [00:00,  6.64it/s]
8it [00:01,  6.51it/s]
9it [00:01,  6.46it/s]
11it [00:01,  7.57it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/bab4f83751a14be7ba6d757674024de4a2c043556a51039eb4708e8b5de0683d_2892.profile
Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/67df737fad5c83781f20c9485749ef999f88e012c1ecdadabf1c8ab3b8f14a00_432.profile



13it [00:01,  8.50it/s]
15it [00:01,  9.29it/s]

...

Empty profile: ./data/tree-rep-profiles_o2o/EXP/virut/G0/c68c2c0d0d6c9788293c8d22891961fb802887fa7fa06b2ee83e6b8d1737e5af_3332.profile



3860it [08:38,  7.91it/s]
3862it [08:39,  8.94it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/virut/G0/142f7bedf10ef234b62fbf9a20f95e0c19e00c9a4a6f983c397515a18c999abf_3120.profile



3863it [08:39,  8.29it/s]
3864it [08:39,  7.36it/s]
3865it [08:39,  7.36it/s]
3866it [08:39,  6.95it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

3867it [08:39,  7.52it/s]
3868it [08:39,  6.80it/s]
3869it [08:40,  6.91it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

3871it [08:40,  7.17it/s]
3872it [08:40,  6.87it/s]
3873it [08:40,  6.67it/s]
3874it [08:40,  6.73it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: Runt

ExactMatchAccuracyScore: 0.07257799671592775 EM_hash#:/all_hash#:/NoIndREP_sample#: 221 3045 0 ExactMatchPrecision(w/o No ind REP): 0.07257799671592775 TopNAccScore: 0.9130901287553648 TopNAccScore(w/o No ind REP): 0.9130901287553648


ExactMatchAccuracyScore: 0.07257799671592775 EM_hash#:/all_hash#:/NoIndREP_sample#: 221 3045 0 ExactMatchPrecision(w/o No ind REP): 0.07257799671592775 TopNAccScore: 0.9477825464949928 TopNAccScore(w/o No ind REP): 0.9477825464949928

(0.8)

In [146]:
basic_statistics(temp[0])

,length
mean,0.934728
std,0.078276
mode,0.000000
min,0.257708
q1,0.919924
median,0.955627
q3,0.975992
max,1.000000
iqr,0.056068
outlier,1.060095


## Loner kNN
* align with loner dir encode

In [161]:
loner_dir = './data/tree-rep-profiles_o2o/EXP/'
fam_exp = next(os.walk(loner_dir))[1]
encode_fam = {}
for i,v in enumerate(fam_exp):
    encode_fam[v] = i
pickle.dump(file=open('data/tree-rep-profiles_o2o/encode_fam.pkl','wb'),obj=encode_fam)
encode_fam

{'parite': 0,
 'ipamor': 1,
 'yantai': 2,
 'outbrowse': 3,
 'megasearch': 4,
 'autoit': 5,
 'hotbar': 6,
 'lydra': 7,
 'directdow': 8,
 'solimba': 9,
 'domaiq': 10,
 'eggnog': 11,
 'zusy': 12,
 'picsys': 13,
 'sirefef': 14,
 'browsefox': 15,
 'zbot': 16,
 'ursu': 17,
 'kazy': 18,
 'sytro': 19,
 'pcclient': 20,
 'shodi': 21,
 'allaple': 22,
 'graftor': 23,
 'katusha': 24,
 'mira': 25,
 'fesber': 26,
 'razy': 27,
 'morstar': 28,
 'expiro': 29,
 'fakeav_fakealert': 30,
 'sality': 31,
 'symmi': 32,
 'softpulse': 33,
 'vilsel': 34,
 'bdmj': 35,
 'ramnit': 36,
 'installerex': 37,
 'vobfus': 38,
 'somoto': 39,
 'clickdownload': 40,
 'downloa_soft': 41,
 'mplug': 42,
 'medfos': 43,
 'virut': 44}

In [165]:
# train data prepare
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/' #要有最後的反斜線
ind_reps = next(os.walk(ind_rep_dir))[2]
kNN_train_Y = []
for i,c in enumerate(ind_reps):
    ind_rep = pickle.load(open(ind_rep_dir + c,'rb')) #每個tree的centroid pickle
    fam_name = c.split('_')[0].split('.')[-1]
    for j,v in enumerate(list(encode_fam.keys())):
        if fam_name in v:
            kNN_train_Y.append(j)
            break
    if i==0:
        kNN_train_X = ind_rep # 1st time
    else:
        kNN_train_X = np.concatenate((kNN_train_X,ind_rep),axis=0) #依順序合在一起 
assert kNN_train_X.shape[0] == len(kNN_train_Y)

Validation dataset to choose k

In [192]:
def data_prepare(inf_pid_paths,inf_emb_api,inf_emb,encode_fam=encode_fam,loner_dir=False):
    em_up = 0
    no_ind_rep = 0
    inf_X = []
    inf_Y = []
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        if loner_dir:
            fam_name = path.split('/')[-3]
        else:
            fam_name = path.split('/')[-3].split('_')[0].split('.')[-1]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        try:
            assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
            assert api[:length+1][-1] == 0
        except:
            print('===Assertion ERR:',i,path)
            continue
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
#                 no_ind_rep_hash.append(recent_hash)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        inf_X.append(rep_emb_avg)
        for j,v in enumerate(list(encode_fam.keys())):
            if fam_name in v:
                inf_Y.append(j)
                break        
    assert len(inf_X) == len(inf_Y)
    return np.array(inf_X),inf_Y

In [ ]:
valid_X , valid_Y = data_prepare(valid_pid_paths,valid_emb_api,valid_emb)

In [191]:
for i in [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31]:
    neigh = KNeighborsClassifier(n_neighbors=i,n_jobs=30,weights='distance') #,weights='distance'
    neigh.fit(kNN_train_X, kNN_train_Y) 
    score = neigh.score(valid_X,valid_Y)
    print('k=',i,',acc=',score)

k= 3 ,acc= 0.7404129793510325
k= 5 ,acc= 0.7522123893805309
k= 7 ,acc= 0.7522123893805309
k= 9 ,acc= 0.7610619469026548
k= 11 ,acc= 0.7551622418879056
k= 13 ,acc= 0.7581120943952803
k= 15 ,acc= 0.7551622418879056
k= 17 ,acc= 0.7551622418879056
k= 19 ,acc= 0.7551622418879056
k= 21 ,acc= 0.7581120943952803
k= 23 ,acc= 0.7581120943952803
k= 25 ,acc= 0.7581120943952803
k= 27 ,acc= 0.7581120943952803
k= 29 ,acc= 0.7581120943952803
k= 31 ,acc= 0.7581120943952803


In [ ]:
exp_X , exp_Y = data_prepare(exp_pid_paths,exp_api,exp_emb)

In [200]:
neigh = KNeighborsClassifier(n_neighbors=1311,n_jobs=30) #,weights='distance'
neigh.fit(kNN_train_X, kNN_train_Y) 
score = neigh.score(exp_X,exp_Y)
ans = neigh.predict(exp_X)
score

0.06925465838509316

In [198]:
ans

array([43, 30, 30, ..., 30, 39,  9])

# Deprecated

## Train
* cluster each tree
* avg vector
* threashold
* 改成跟每個call比較距離?
* 改成cosine sim?

In [168]:
fam_name = None
tree_name = None
tree_vec = []
root_dir = './data/tree-rep-profiles_o2o/centroids/'
if not os.path.exists(root_dir):
    os.makedirs(root_dir,exist_ok=True)
for path,api,emb in tqdm(zip(train_pid_paths,train_emb_api,train_emb)):
    recent_fam = path.split('/')[-3].split('_')[0]
    recent_tree = path.split('/')[-2]
    with open(path,encoding='ISO 8859-1') as f: #X2
        lines = f.read()
    lines = re.sub(r'[^\x00-\x7F]+','', lines)
    lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
    lines = re.sub('=\\n','',lines)
    lines = re.sub('y\\n','',lines)
    lines = lines.splitlines()
    length = len(lines)
    if length < 11:
        print('No common rep. Should not in a tree:',path)
        continue
#     print(len(lines))
    assert api[:length][-1] != 0
    assert api[:length+1][-1] == 0
    api_e = np.expand_dims(api, axis=0)
    emb_e = np.expand_dims(emb, axis=0)
    final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
    final_emb = final_emb_e[0][:length] #(length,768)
    byte_rep_e = model.predict([api_e,emb_e])
    byte_rep = byte_rep_e[0][:length] # (length,1)
    byte_rep_thr = []
    for num in byte_rep:
        if num[0] < thr:
            byte_rep_thr.append(0)
        else:
            byte_rep_thr.append(1) # high than threshold, keep it
    byte_rep_thr = np.array(byte_rep_thr)
    byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
    mul_emb = np.multiply(final_emb,byte_rep_thr)
    rep_emb = [] #超過threshold的emb才留下來
    for emb in mul_emb:
        if (sum(emb) <0) or (sum(emb)>0):
            rep_emb.append(emb)
    rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree
    if recent_fam==fam_name and recent_tree==tree_name:
        tree_vec.append(rep_emb_avg) #如果同tree同fam就把rep_emb加進去list
    else:
        if tree_vec:
            fam_tree_c = np.average(tree_vec,axis=0) # 求出整個tree的centroid
            pickle.dump(file=open(root_dir+fam_name+'_'+tree_name+'.pkl','wb'),obj=fam_tree_c)
            tree_vec = []
        fam_name = recent_fam
        tree_name = recent_tree
        if not tree_vec: # if tree vector list is empty
            tree_vec.append(rep_emb_avg)
#     break
#     print(path,api,emb)
#     break

114it [00:17,  7.03it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1538it [03:46,  7.47it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3372.profile


1545it [03:47, 10.52it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/1cd7a58cb78d893a0c943547afd917fd446cdc16a9259286062c7972dadeba77_3300.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/12306d8f978ba2830dc207d05c9b1963edacca2d10a935654cbdcecc3e744b7b_3312.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3620.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3276.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3412.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5

1937it [04:44,  8.17it/s]

...

9750it [22:34,  8.02it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/21.soft_0.8/G222/4c6cbd4288806a30edc643a4d4bdbbc80d6a452680ed74f4be2ac2ff6a2d063a_3352.profile


9765it [22:36,  8.20it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/21.soft_0.8/G222/0faf56b2177f5b411e4a8dd077634b0988e32b54bb47b0da7e3de0cd86412df7_3220.profile


11455it [26:48,  5.94it/s]


## Dev / Test
* calculate euclidean distance
* check if match right tree/fam
* top1/top3/top5
* 同sample不同pid的歸類，同分的歸類 (?)
* topN dis<0.2以下的歸類 (?)
* family改成是最近的3/5個family (同family就往下推移)
* 把training沒有此tree的DEV pid拿掉

In [257]:
centroid_dir = './data/tree-rep-profiles_o2o/centroids/' #要有最後的反斜線
def inf_evaluate(inf_pid_paths,inf_emb_api,inf_emb,centroid_dir=centroid_dir):
    centroids = next(os.walk(centroid_dir))[2]
    centroid_matrix = None
    c_fam = []
    c_tree = []
    err_centroid = []
    top1_fam_up = 0
    top1_tree_up = 0
    top3_fam_up = 0
    top3_tree_up = 0
    top5_fam_up = 0
    top5_tree_up = 0
    wrong_fam = {}
    wrong_tree = {}
    no_ind_rep = 0
    for i,c in enumerate(centroids):
        tree_rep = pickle.load(open(centroid_dir + c,'rb')) #每個tree的centroid pickle
        tree_rep = np.expand_dims(tree_rep,axis=0)
        try:
            chk = sum(sum(tree_rep))
        except TypeError:
            err_centroid.append(c)
            continue
        if (not chk < 0) and (not chk>0):
            err_centroid.append(c)
            continue
        if i >0:
            centroid_matrix = np.concatenate((centroid_matrix,tree_rep),axis=0) #依順序合在一起       
        if i==0:
            centroid_matrix = tree_rep # 1st time
        c_fam.append(c.split('_')[0]) # centroid的fam name
        c_tree.append(c.split('_')[-1].split('.')[0]) # centroid的tree name
    assert centroid_matrix.shape[0] == len(c_fam) == len(c_tree)
    assert len(inf_pid_paths) == len(inf_emb_api) == len(inf_emb)
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e])
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr) #把embediing跟超過threshold的byterep相乘  
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if sum(emb) !=0: #跟0鄉城都會=0
                rep_emb.append(emb)       
        rep_emb_avg = np.average(rep_emb,axis=0)
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        dist = euclidean_distances(rep_emb_avg,centroid_matrix) #取出與各rep距離，求min的idx對答案
        min_dist = np.min(dist) #top1
        min_idx = np.argmin(dist) #top1
        dist5 = dist[0].argsort()[:5] #top3
        if c_fam[min_idx] == recent_fam:
            top1_fam_up = top1_fam_up+1
        else:
            wrong_fam[path] = c_fam[min_idx]
        if (c_fam[min_idx] == recent_fam) and (c_tree[min_idx] == recent_tree):
            top1_tree_up = top1_tree_up+1
        else:
            wrong_tree[path] = c_fam[min_idx]+'_'+ c_tree[min_idx]
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]:
            top3_fam_up = top3_fam_up+1
        if (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]]]) and (recent_fam in[c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]):
            top3_tree_up = top3_tree_up+1
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]:
            top5_fam_up = top5_fam_up+1
        if (recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]) and (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]],c_tree[dist5[3]],c_tree[dist5[4]]]):
            top5_tree_up = top5_tree_up + 1
    down = len(inf_pid_paths)
    print('Top1 Fam score:',top1_fam_up/down,'Top1 Tree score:',top1_tree_up/down,
         'Top3 Fam score:',top3_fam_up/down,'Top3 Tree score:',top3_tree_up/down,
          'Top5 Fam score:',top5_fam_up/down,'Top5 Tree score:', top5_tree_up/down,
          'No ind rep#:',no_ind_rep
         )
    return wrong_fam,wrong_tree
#         for c in centroids:
#             tree_rep = pickle.load(open(centroid_dir + c,'rb'))


In [258]:
temp = inf_evaluate(test_pid_paths,test_emb_api,test_emb)


0it [00:00, ?it/s]
1it [00:00,  6.89it/s]
2it [00:00,  6.92it/s]
3it [00:00,  6.19it/s]
5it [00:00,  6.89it/s]
6it [00:00,  7.23it/s]
7it [00:00,  7.36it/s]
8it [00:01,  7.51it/s]
9it [00:01,  7.71it/s]
10it [00:01,  7.70it/s]
11it [00:01,  8.07it/s]
12it [00:01,  8.40it/s]
13it [00:01,  7.97it/s]
14it [00:01,  8.01it/s]
15it [00:01,  8.14it/s]
16it [00:02,  7.88it/s]
17it [00:02,  7.73it/s]
18it [00:02,  7.58it/s]
19it [00:02,  7.63it/s]
20it [00:02,  7.49it/s]
21it [00:02,  7.59it/s]
22it [00:02,  7.78it/s]
23it [00:02,  8.04it/s]
24it [00:03,  8.20it/s]
25it [00:03,  8.26it/s]
26it [00:03,  8.43it/s]
27it [00:03,  8.36it/s]
28it [00:03,  8.20it/s]
29it [00:03,  7.97it/s]
30it [00:03,  8.00it/s]
31it [00:03,  7.99it/s]
32it [00:04,  7.92it/s]
33it [00:04,  7.95it/s]
34it [00:04,  7.94it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/c

Empty profile: ./data/tree-rep-profiles_o2o/TEST/27.downloa_0.8/G184/0f3dbceacbbebc3dd86d6f7c24f964f8c20a46d40c265701cb2ce2b557858188_3284.profile


/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

166it [00:20,  9.31it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

168it [00:21,  9.07it/s]
169it [00:21,  8.69it/s]
170it [00:21,  8.33it/s]
171it [00:21,  8.24it/s]
172it [00:21,  7.91it/s]
173it [00:21,  7.76it/s]
174it [00:21,  7.79it/s]
175it [00:22,  7.78it/s]
176it [00:22,  7.78it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  a

Empty profile: ./data/tree-rep-profiles_o2o/TEST/27.downloa_0.8/G208/0f3dbceacbbebc3dd86d6f7c24f964f8c20a46d40c265701cb2ce2b557858188_3284.profile



225it [00:28,  9.22it/s]
226it [00:28,  8.70it/s]
228it [00:28,  9.92it/s]

...


655it [01:20,  8.88it/s]
656it [01:20,  8.46it/s]
657it [01:20,  8.35it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

658it [01:20,  8.49it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

660it [01:21,  8.81it/s]
661it [01:21,  8.93it/s]
662it [01:21,  8.85it/s]
663it [01:21,  8.70it/s]
664it [01:21,  8.16it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/l

755it [01:32,  8.13it/s]
757it [01:32,  8.28it/s]
758it [01:33,  7.94it/s]
759it [01:33,  8.27it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

760it [01:33,  8.28it/s]
761it [01:33,  8.40it/s]
762it [01:33,  8.03it/s]
763it [01:33,  8.16it/s]
764it [01:33,  8.26it/s]
765it [01:33,  7.62it/s]
766it [01:34,  7.70it/s]
767it [01:34,  8.00it/s]
768it [01:34,  7.86it/s]
769it [01:34,  8.08it/s]
770it [01:34,  8.04it/s]
771it [01:34,  8.06it/s]
772it [01:34,  8.09it/s]
773it [01:34,  7.66it/s]
774it [01:35,  7.49it/s]
775it [01:35,  7.66it/s]
776it [01:35,  7.70it/s]
778it [01:35,  8.04it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axi

Empty profile: ./data/tree-rep-profiles_o2o/TEST/21.soft_0.8/G199/4c6cbd4288806a30edc643a4d4bdbbc80d6a452680ed74f4be2ac2ff6a2d063a_3508.profile



851it [01:44,  9.62it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

853it [01:44,  9.63it/s]
855it [01:44,  8.99it/s]
856it [01:44,  8.92it/s]
857it [01:44,  8.74it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

858it [01:44,  8.71it/s]
860it [01:45,  8.53it/s]
861it [01:45,  8.52it/s]
862it [01:45,  8.69it/s]
863it [01:45,  8.57it/s]
864it [01:45,  8.37it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning:

Top1 Fam score: 0.060114503816793896 Top1 Tree score: 0.04484732824427481 Top3 Fam score: 0.14026717557251908 Top3 Tree score: 0.10019083969465649 Top5 Fam score: 0.18129770992366412 Top5 Tree score: 0.1364503816793893 No ind rep#: 95


## Loner EXP classification

In [269]:
centroid_dir = './data/tree-rep-profiles_o2o/centroids/' #要有最後的反斜線
def loner_evaluate(lon_pid_paths,lon_emb_api,lon_emb,centroid_dir=centroid_dir):
    centroids = next(os.walk(centroid_dir))[2]
    centroid_matrix = None
    c_fam = []
    c_tree = []
    err_centroid = []
    top1_fam_up = 0
#     top1_tree_up = 0
    top3_fam_up = 0
#     top3_tree_up = 0
    top5_fam_up = 0
#     top5_tree_up = 0
    wrong_fam = {}
#     wrong_tree = {}
    no_ind_rep = 0
    for i,c in enumerate(centroids):
        tree_rep = pickle.load(open(centroid_dir + c,'rb')) #每個tree的centroid pickle
        tree_rep = np.expand_dims(tree_rep,axis=0)
        try:
            chk = sum(sum(tree_rep))
        except TypeError:
            err_centroid.append(c)
            continue
        if (not chk < 0) and (not chk>0):
            err_centroid.append(c)
            continue
        if i >0:
            centroid_matrix = np.concatenate((centroid_matrix,tree_rep),axis=0) #依順序合在一起       
        if i==0:
            centroid_matrix = tree_rep # 1st time
        c_fam.append(c.split('_')[0]) # centroid的fam name
#         c_tree.append(c.split('_')[-1].split('.')[0]) # centroid的tree name
    assert centroid_matrix.shape[0] == len(c_fam) #== len(c_tree)
    assert len(lon_pid_paths) == len(lon_emb_api) == len(lon_emb)
    for path,api,emb in tqdm(zip(lon_pid_paths,lon_emb_api,lon_emb)):
        recent_fam = path.split('/')[-3].split('_')[0]
#         recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e])
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr) #把embediing跟超過threshold的byterep相乘  
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if sum(emb) !=0: #跟0鄉城都會=0
                rep_emb.append(emb)       
        rep_emb_avg = np.average(rep_emb,axis=0)
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):#沒有individual REP
                no_ind_rep+=1
                continue 
        except ValueError: #正常profile
            pass
        dist = euclidean_distances(rep_emb_avg,centroid_matrix) #取出與各rep距離，求min的idx對答案
        min_dist = np.min(dist) #top1
        min_idx = np.argmin(dist) #top1
        dist5 = dist[0].argsort()[:5] #top3
        if c_fam[min_idx] == recent_fam:
            top1_fam_up = top1_fam_up+1
        else:
            wrong_fam[path] = c_fam[min_idx]
#         if (c_fam[min_idx] == recent_fam) and (c_tree[min_idx] == recent_tree):
#             top1_tree_up = top1_tree_up+1
#         else:
#             wrong_tree[path] = c_fam[min_idx]+'_'+ c_tree[min_idx]
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]:
            top3_fam_up = top3_fam_up+1
#         if (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]]]) and (recent_fam in[c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]):
#             top3_tree_up = top3_tree_up+1
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]:
            top5_fam_up = top5_fam_up+1
#         if (recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]) and (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]],c_tree[dist5[3]],c_tree[dist5[4]]]):
#             top5_tree_up = top5_tree_up + 1
    down = len(lon_pid_paths)
    print('Top1 Fam score:',top1_fam_up/down,#'Top1 Tree score:',top1_tree_up/down,
         'Top3 Fam score:',top3_fam_up/down,#'Top3 Tree score:',top3_tree_up/down,
          'Top5 Fam score:',top5_fam_up/down,#'Top5 Tree score:', top5_tree_up/down,
          'No ind rep#:',no_ind_rep
         )
    return wrong_fam
#         for c in centroids:
#             tree_rep = pickle.load(open(centroid_dir + c,'rb'))


In [270]:
temp = loner_evaluate(exp_pid_paths,exp_api,exp_emb)


0it [00:00, ?it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

1it [00:00,  6.42it/s]
2it [00:00,  7.10it/s]
3it [00:00,  7.50it/s]
4it [00:00,  7.71it/s]
5it [00:00,  7.90it/s]
6it [00:00,  7.91it/s]
7it [00:00,  8.06it/s]
8it [00:00,  8.24it/s]
9it [00:01,  7.99it/s]
10it [00:01,  8.02it/s]
11it [00:01,  8.05it/s]
12it [00:01,  8.12it/s]
13it [00:01,  8.15it/s]
14it [00:01,  8.01it/s]
15it [00:01,  8.01it/s]
16it [00:01,  7.99it/s]
17it [00:02,  7.95it/s]
18it [00:02,  7.75it/s]
19it [00:02,  7.89it/s]
20it [00:02,  8.01it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/

124it [00:15,  8.39it/s]
125it [00:15,  8.46it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

126it [00:15,  8.55it/s]
127it [00:15,  8.71it/s]
128it [00:15,  8.42it/s]
129it [00:15,  8.27it/s]
130it [00:15,  7.94it/s]
131it [00:15,  7.71it/s]
132it [00:16,  7.97it/s]
133it [00:16,  8.19it/s]
134it [00:16,  8.27it/s]
135it [00:16,  8.26it/s]
136it [00:16,  8.31it/s]
137it [00:16,  8.29it/s]
138it [00:16,  8.20it/s]
139it [00:16,  8.13it/s]
140it [00:17,  8.11it/s]
141it [00:17,  7.88it/s]
142it [00:17,  7.72it/s]
143it [00:17,  7.68it/s]
144it [00:17,  7.27it/s]
145it [00:17,  6.88it/s]
146it [00:17,  6.91it/s]
147it [00:18,  6.93it/s]
148it [00:18,  7.03it/s]
149it [00:18,  6.95it/s]
150it [00:18,  7.05it/s]
151it [00:1

291it [00:36,  8.14it/s]
292it [00:36,  8.56it/s]
293it [00:36,  8.55it/s]
294it [00:36,  8.37it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

295it [00:36,  8.52it/s]
297it [00:36,  8.63it/s]
298it [00:36,  8.43it/s]
299it [00:37,  8.31it/s]
300it [00:37,  8.32it/s]
301it [00:37,  8.11it/s]
302it [00:37,  7.74it/s]
303it [00:37,  7.40it/s]
304it [00:37,  7.58it/s]
305it [00:37,  7.42it/s]
306it [00:37,  7.42it/s]
307it [00:38,  7.52it/s]
308it [00:38,  7.90it/s]
309it [00:38,  7.97it/s]
310it [00:38,  8.00it/s]
311it [00:38,  8.09it/s]
312it [00:38,  8.11it/s]
313it [00:38,  8.27it/s]
314it [00:38,  8.04it/s]
315it [00:39,  8.03it/s]
316it [00:39,  8.14it/s]
317it [00:39,  8.20it/s]
318it [00:39,  8.13it/s]
319it [00:3

Top1 Fam score: 0.17045454545454544 Top3 Fam score: 0.29545454545454547 Top5 Fam score: 0.34375 No ind rep#: 37


NameError: name 'wrong_tree' is not defined

In [264]:
centroids[0].split('_')[0]

'2.zbot'

In [236]:
dist_s = dist[0].argsort()[:3]

In [238]:
dist_s[0]

118

In [228]:
np.argmin(dist)

118

In [227]:
dist[0][118]

2.3757918363134127

In [214]:
rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)

In [210]:
i

830

In [188]:
tree_rep = pickle.load(open(centroid_dir + '20.ramnit_G420.pkl','rb'))
tree_rep

nan

In [218]:
centroid_matrix.shape

(729, 768)

In [230]:
recent_tree

'G22'

In [164]:
tree_rep = pickle.load(open(root_dir + '22.razy_0.8_G281.pkl','rb'))
kkkk = np.expand_dims(tree_rep,axis=0)
# tree_rep = 
kkkk

array([[ 8.65596963e-02, -1.38961762e+00, -8.80325557e-01,
         1.44868998e+00,  3.33986451e+00,  1.84224879e+00,
        -1.22117380e+00, -1.73912917e+00, -2.14322432e+00,
        -2.42802131e+00,  7.90877797e-01, -9.03681114e-01,
        -1.33191333e+00, -1.42146935e+00, -3.65049951e+00,
        -3.09185951e-01,  2.77627880e+00,  3.25421317e+00,
         1.27467083e+00, -2.09883721e-01,  1.37831007e+00,
        -1.50192061e+00,  1.86882590e-01,  2.07811224e+00,
        -3.61454106e+00, -2.11806000e+00,  1.17014932e-01,
        -9.56569921e-01, -3.29551819e-01, -7.14762268e-01,
        -6.92955113e-01,  2.05578955e+00,  2.02854938e+00,
         1.70297706e+00, -1.78908776e+00,  1.28335643e+00,
        -5.62766004e+00,  2.14058878e-01,  6.18024734e-02,
         2.52637649e+00,  2.05098724e-01,  3.90071236e-01,
         4.08042537e-01, -2.85100074e-01,  1.17381613e+00,
         3.74853606e+00, -1.58441165e+00,  9.09052398e-01,
         2.28420114e+00,  2.40741630e+00, -5.56467452e-0

In [169]:
k = np.expand_dims(rep_emb_avg,axis=0)
kk = np.repeat(k,5,axis=0)
k

array([[-2.32679514e-03, -1.51152636e+00, -1.32905939e+00,
         9.56873546e-01,  3.48319108e+00,  1.81062158e+00,
        -1.19937576e+00, -1.57059526e+00, -2.14856078e+00,
        -2.66518752e+00,  2.08959677e-01, -1.13687739e+00,
        -1.03918429e+00, -6.80092806e-01, -3.51498023e+00,
        -2.18805710e-01,  2.73317995e+00,  3.09727030e+00,
         1.15348836e+00, -1.18641081e+00,  1.43051075e+00,
        -1.39607588e+00,  7.68887893e-01,  2.14773534e+00,
        -3.52791246e+00, -1.77533390e+00, -7.99679553e-02,
        -5.23277587e-01, -7.02902425e-01, -8.52432431e-01,
        -4.29765171e-01,  2.15343165e+00,  1.83647086e+00,
         6.52459159e-01, -1.55454592e+00,  1.03299757e+00,
        -4.88682050e+00,  2.14824384e-02,  4.52365744e-01,
         3.43573372e+00,  3.48641196e-01,  6.86164492e-01,
         4.88668572e-01, -7.38335954e-01,  7.35207275e-01,
         3.97278129e+00, -7.10923588e-01, -1.35453728e-01,
         2.37253270e+00,  2.09358406e+00, -7.30818144e-0

In [135]:
rep_emb_avg.shape

(768,)

In [167]:
k = None
np.concatenate((kkkk,k),axis=0)

ValueError: all the input arrays must have same number of dimensions

In [153]:
euclidean_distances(kk,k)

array([[12.34622314],
       [12.34622314],
       [12.34622314],
       [12.34622314],
       [12.34622314]])

In [131]:
next(os.walk(root_dir))[2]

['22.razy_0.8_G281.pkl',
 '19.installcore_0.8_G344.pkl',
 '139.softonic_0.8_G34.pkl',
 '13.vobfus_0.8_G462.pkl',
 '2.zbot_0.8_G1370.pkl',
 '25.browsefox_0.8_G309.pkl',
 '6.virut_0.8_G1418.pkl',
 '37.somoto_0.8_G215.pkl',
 '4.kazy_0.8_G1242.pkl',
 '9.loadmoney_0.8_G614.pkl',
 '16.sytro_0.8_G485.pkl',
 '29.parite_0.8_G298.pkl',
 '11.zusy_0.8_G556.pkl',
 '52.megasearch_0.8_G132.pkl',
 '25.browsefox_0.8_G304.pkl',
 '5.ursu_0.8_G787.pkl',
 '7.solimba_0.8_G740.pkl',
 '57.vilsel_0.8_G433.pkl',
 '3.allaple_0.8_G1429.pkl',
 '39.sirefef_0.8_G119.pkl',
 '54.fesber_0.8_G351.pkl',
 '10.symmi_0.8_G609.pkl',
 '2.zbot_0.8_G1355.pkl',
 '187.wenper_0.8_G17.pkl',
 '32.outbrowse_0.8_G266.pkl',
 '2.zbot_0.8_G1317.pkl',
 '59.installrex_0.8_G96.pkl',
 '5.ursu_0.8_G999.pkl',
 '26.autoit_0.8_G256.pkl',
 '3.allaple_0.8_G2028.pkl',
 '141.peed_0.8_G19.pkl',
 '21.soft_0.8_G250.pkl',
 '5.ursu_0.8_G1035.pkl',
 '124.mediafinder_0.8_G18.pkl',
 '4.kazy_0.8_G1252.pkl',
 '13.vobfus_0.8_G447.pkl',
 '20.ramnit_0.8_G516.pkl